**Name**: Michelle Adeline

**Student ID**: 31989101

### Download required packages

Please uncomment the commands below to install the dependencies if you do not have the required packages/files.

In [21]:
# !pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
# !pip install matplotlib numpy pandas tqdm nltk scikit-learn

# for separating ingredients vs non-ingredients
# NOTE: if using Windows to run this, need to download GNU Wget
# !wget -c https://raw.githubusercontent.com/williamLyh/RecipeWithPlans/main/ingredient_set.json -O ingredient_set.json

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.6 kB ? eta -:--:--
     -------------------------------------- 60.6/60.6 kB 811.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/11.0 MB 7.2 MB/s eta 0:00:02
   --- ------------------------------------ 1.0/11.0 MB 10.2 MB/s eta 0:00:01
   ----- ---------------------------------- 1.6/11.0 MB 11.3 MB/s eta 0:00:01
   -------- ------------------------------- 2.2/11.0 MB 11.7 MB/s eta 0:00:01
   ---------- ----------------------------- 2.8/11.0 MB 12.1 MB/s eta 0:00:01
   ----------- ---------------------------- 3.1/11.0 MB 11.7 MB/s eta 0:00:01
   ------------- -------------------------- 3.6/11.0 MB 11.5 MB/s eta 0:00:01
   -------------- ------------------------- 4.1/11.0 MB 11.4 MB/s eta 0:00:01
   ----------------- ---------------------- 4.7/11.0 MB 11.5 MB/s eta 0:00:01
   -


nuscenes-devkit 1.1.10 requires opencv-python, which is not installed.
nuscenes-devkit 1.1.10 requires pycocotools>=2.0.1, which is not installed.
nuscenes-devkit 1.1.10 requires Shapely<=1.8.5, which is not installed.
nuscenes-devkit 1.1.10 requires matplotlib<=3.5.2, but you have matplotlib 3.8.4 which is incompatible.


### Imports and Constants

In [22]:
import os
import time
import math
import re
import string
import numpy as np
import pandas as pd
import random
import json
from copy import deepcopy

import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR, MultiStepLR, CosineAnnealingWarmRestarts
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
from torch import nn, optim

import nltk
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate import meteor
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

nltk.download("wordnet")
nltk.download("punkt")
nltk.download('stopwords')

SEED = 31989101
HIDDEN_SIZE = 256
MAX_INGR_LEN = 150 # fixed from assignment
MAX_RECIPE_LEN = 600 # empirically set
DROPOUT = 0.1
TEACHER_FORCING_RATIO = 1.0 # fixed from assignment
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ING_START = "<INGREDIENT_START>"
ING = "<INGREDIENT>"
ING_END = "<INGREDIENT_END>"
REC_START = "<RECIPE_START>"
REC = "<RECIPE_STEP>"
REC_END = "<RECIPE_END>"
SPECIAL_TAGS = {
    ING_START: 0,
    ING_END: 1,
    REC_START: 2,
    REC_END: 3,
    ING: 4,
    REC: 5,
}

PAD_WORD = "<PAD>"
UNKNOWN_WORD = "<UNKNOWN>"

## ensuring reproducibility
def reset_rng():
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)

reset_rng()

# to easily read ingredients and instructions
pd.set_option('display.max_colwidth', 2000)

print(f"Using device: {DEVICE}")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...


Using device: cpu


[nltk_data]   Unzipping corpora\stopwords.zip.


# Data Loading and Preprocessing

### Data Exploration

In [3]:
data_root = "./Cooking_Dataset"
add_intermediate_tag=False

train_df_orig = pd.read_csv(os.path.join(data_root, "train.csv"), usecols=['Ingredients', 'Recipe'])
dev_df_orig = pd.read_csv(os.path.join(data_root, "dev.csv"), usecols=['Ingredients', 'Recipe'])
test_df_orig = pd.read_csv(os.path.join(data_root, "test.csv"), usecols=['Ingredients', 'Recipe'])

In [4]:
train_df_orig.head()

,Ingredients,Recipe
0,6 tb butter or margarine - softened\t3/4 c c and h powdered sugar\t1 c all-purpose flour\t1 tb milk\t2 eggs\t1 c c and h granulated sugar\t1/2 c cocoa\t2 tb flour\t1/2 ts baking powder\t1/2 ts salt\t1 ts vanilla\t1/4 ts almond extract (optional)\t1 c chopped almonds or pecans,"cream together butter and powdered sugar . blend in 1 cup flour and milk . spread evenly in bottom of ungreased 9-inch square pan . bake in 350 degree oven 10 to 12 minutes . beat eggs slightly ; combine dry ingredients and add to eggs . blend in vanilla and almond extract ; fold in almonds . spread over hot baked layer ; return to oven and bake 20 minutes longer . cool ; while warm , cut into 24 bars ."
1,"1 c vanilla wafer cookies, finely crushed\t1 c confectioner's sugar\t2 tb cocoa\t1 c pecans, finely chopped\t2 tb corn syrup\t1/2 c bourbon","combine dry ingredients and mix well . add corn syrup and bourbon and mix well . if too moist add a few cookie crumbs , if too dry add a little more bourbon . shape into small balls and roll in fruit sugar or fonely chopped pecans . store in an airtight container . also good made with dark rum ."
2,1 c all-purpose flour\t1/2 c sugar\t2 ts baking powder\t1/2 ts salt\t2 tb unsweetened cocoa powder\t1 1/2 c all-bran? cereal\t3/4 c skim milk\t2 egg whites\t1/4 c vegetable oil\t1 c sliced ripe banana; (about 1\tvegetable cooking spray,"kellogg 's all-bran 1 . stir together flour , sugar , baking powder , salt , and cocoa powder . set aside . in large mixing bowl , combine all-bran cereal and milk . let stand 5 minutes or until cereal softens . add egg whites and oil . beat well . stir in bananas . add flour mixture stirring only until combined . portion batte evenly into twelve 2-1/2 inch muffin-pan cups coated with cooking spray . bake at 400 f. about 25 minutes or until lightly browned . serve warm ."
3,"5 1/2 c flour\t1 cake compressed yeast\tor 1 cake dry yeast\t2 c milk, scalded and cooled\t1/2 ts salt\t1/2 c cocoa\t1/4 c shortening\t2 eggs, well beaten\t1/2 c sugar","soften yeast and dissolve 1 tablespoon sugar in lukewarm milk . add 3 cups flour and beat until smooth . cover and set aside to rise in a warm place until light . add shortening and sugar , which have been creamed together , eggs , cocoa , salt , and remainder of flour or enough to make a soft dough . knead lightly and place in well-oiled bowl . cover and set in a warm place until double in bulk about 2 hours . form into loaves . place in well-oiled bread pans , filling them 1/2 full . cover and let rise again until double in bulk . bake in hot oven -lrb- 425 f -rrb- 40 50 minutes . 2 loaves ."
4,bread -- to fill dish 2/3\tful\t2 c skim milk\t2/3 c sugar\t2 eggs -- or 4 egg whites\t2 tb cocoa\t1 ts vanilla,"into a greased casserole dish break up enough bread to fill it 2/3 full . over this pour 2/3 cup sugar , 2 heaping tbsp cocoa that have been mixed together . toss all together lightly -lrb- to coat the bread -rrb- . to 2 cups of milk add 2 well beaten eggs and 1 tsp vanilla . pour this over the bread and it should just cover the pieces . bake at 350 fo r about 45 minutes ."


`dev` and `test` are all lowercase already. `train` supposedly has 2 rows containing non-lowercase letters. Let's investigate them

Only 2 rows in Ingredients have non lowercase letters but upon investigation, both are invalid rows so can simply remove them.

In [5]:
def check_all_lowercase(df):
    ingrnonlower = (~df.Ingredients.dropna().str.islower()).sum()
    recipenonlower = (~df.Recipe.dropna().str.islower()).sum()
    print(f"Number of rows with a non-lowercase letter:\nIngredients: {ingrnonlower}\nRecipe: {recipenonlower}")

In [6]:
check_all_lowercase(train_df_orig)

Number of rows with a non-lowercase letter:
Ingredients: 2
Recipe: 0


In [7]:
check_all_lowercase(dev_df_orig)

Number of rows with a non-lowercase letter:
Ingredients: 0
Recipe: 0


In [8]:
check_all_lowercase(test_df_orig)

Number of rows with a non-lowercase letter:
Ingredients: 0
Recipe: 0


We can see that the only two rows with non-lowercase letters have invalid Ingredients lists so we can simply filter them out.

In [10]:
train_df_orig[~train_df_orig.Ingredients.str.islower()]

,Ingredients,Recipe
7415,=-=-=-=-=-=-=-=-=-=-=-=-=-=-\t--=-=-=-=-=-= =-=-=-=-=-=-=-=-=-=-=-=-=-=,"follow these 20 `` wellness '' facts and you have a better chance of living a longer and healthier life . michaelangelo was carving the rondanini just before he died at 89 . verdi finished his opera falstaff at 80 . but among non-smokers the risk of one form of lung cancer , adenocarcinoma , increases with the amount of saturated consumed daily . according to the `` new england journal of medicine '' women who have a female physician are twice as likely to receive pap smears . air bags increase the chance of surviving a frontal collision 29 % . about half of adult pedestrians killed in traffic accidents have been drinking . and more than a third were legally drunk . even a two-inch heel increases the pressure to 57 % . half a cantaloupe contains nearly double the amount of vitamin c than an orange . wild animals normally do n't get fat , and their meat is only slightly marbled . try increased fiber and prunes for constipation . researchers believe prunes contain a substance that stimulates intestinal contractions . but if you have asthma , they 're a special hazard . but because it 's `` low-fat '' does n't mean you can eat all you want . so high in calories . growing up is optional"
11772,..........................,"though it 's been around a long time , there 's something almost quintessentially american about the sandwich . after all it was the gambling table that prompted the fourth earl of sandwich -lrb- 1718-1792 -rrb- to have his food served to him between two slices of bread so he would n't have to interrupt his gaming to eat . actually though people were munching on various foods stuffed between slabs of bread long before the earl came into the picture ."


### Getting Dataset Statistics

In [23]:
def get_all_ingredients(fpath, reverse_sort=True):
    # get list of valid ingredients in the dataset from RecipeWithPlans
    with open(fpath, "r") as f:
        all_ings = json.load(f)
    if reverse_sort:
        # sort by longest to shortest ingredients
        all_ings = sorted(all_ings, key=lambda i: len(i.split()), reverse=True)
    return all_ings

def get_ds_statistics(df):
    all_ingredients_list = get_all_ingredients("./ingredient_set.json")

    df_ings_list = df.Ingredients.tolist()
    df_recipes_list = [r for r in df.Recipe.tolist() if not pd.isna(r)]

    print("Tokenizing...")

    tokenized_ings = [word_tokenize(i) for i in df_ings_list]
    tokenized_recipes = [word_tokenize(r) for r in df_recipes_list]

    ings_words = [i.split() for i in df_ings_list]
    recipes_words = [r.split() for r in df_recipes_list]

    ings_lens = [len(i) for i in ings_words]
    recipes_lens = [len(r) for r in recipes_words]

    print("Counting word frequencies...")
    stop_words = set(stopwords.words('english'))

    freqdist_all = [FreqDist() for _ in range(2)]
    freqdist_ing = FreqDist()

    for i, wordset in enumerate([tokenized_ings, tokenized_recipes]):
        for sample in wordset:
            for word in sample:
                if bool(re.match("\w+", word)) and word not in stop_words:
                    freqdist_all[i][word] += 1
                if i == 0 and word in all_ingredients_list:
                    freqdist_ing[word] += 1

    ## count number of ingredients per sample, i.e. how many ingredients are listed for each recipe
    num_ings_per_sample = []

    for i in tokenized_ings:
        num_ings_i = 0
        for word in i:
            num_ings_i += word in all_ingredients_list
        num_ings_per_sample.append(num_ings_i)

    def get_n_gram(n, l):
        cv = CountVectorizer(ngram_range=(n, n)).fit(l)
        bow = cv.transform(l)
        word_sum = bow.sum(axis=0)
        word_freqs = sorted([[word, word_sum[0, idx]] for word, idx in cv.vocabulary_.items()],
                            key=lambda w: w[1], reverse=True)
        return word_freqs
    
    print("Get N-grams...")
    ing_bigrams = get_n_gram(2, df_ings_list)
    ing_trigrams = get_n_gram(3, df_ings_list)

    rec_bigrams = get_n_gram(2, df_recipes_list)
    rec_trigrams = get_n_gram(3, df_recipes_list)

    print(f"Number of samples: {df.shape[0]}")
    print("===== INGREDIENTS =====\n"
          f"min. length: {min(ings_lens)}, max.length: {max(ings_lens)}, avg. length: {sum(ings_lens)/len(ings_lens):.2f}, "
          f"std. length: {np.std(ings_lens):.2f}\n"
          f"Max. number of ingredients: {max(num_ings_per_sample)}, min. number of ingredients: {min(num_ings_per_sample)}, "
          f"avg. number of ingredients: {sum(num_ings_per_sample)/len(num_ings_per_sample)}\n"
          f"10 most common tokens (excluding punuctuation and stopwords): {freqdist_all[0].most_common(10)}\n"
          f"10 most common ingredients: {freqdist_ing.most_common(10)}\n"
          f"Top 5 bigrams: {ing_bigrams[:5]}, top 5 trigrams: {ing_trigrams[:5]}")
    
    print("===== RECIPES =====\n"
          f"min. length: {min(recipes_lens)}, max.length: {max(recipes_lens)}, avg. length: {sum(recipes_lens)/len(recipes_lens):.2f}, "
          f"std. length: {np.std(recipes_lens):.2f}\n"
          f"10 most common tokens (excluding punuctuation and stopwords): {freqdist_all[1].most_common(10)}\n"
          f"Top 5 bigrams: {rec_bigrams[:5]}, top 5 trigrams: {rec_trigrams[:5]}")

In [24]:
# This takes around 3-4 minutes
get_ds_statistics(train_df_orig)

Tokenizing...
Counting word frequencies...
Get N-grams...
Number of samples: 101340
===== INGREDIENTS =====
min. length: 1, max.length: 426, avg. length: 42.25, std. length: 21.04
Max. number of ingredients: 81, min. number of ingredients: 0, avg. number of ingredients: 11.025054272745214
10 most common tokens (excluding punuctuation and stopwords): [('1', 331593), ('c', 272937), ('ts', 167667), ('2', 165295), ('1/2', 164230), ('tb', 133253), ('1/4', 84396), ('chopped', 69067), ('salt', 59032), ('3', 56221)]
10 most common ingredients: [('salt', 59032), ('pepper', 47871), ('sugar', 46062), ('oil', 34928), ('butter', 32738), ('flour', 31032), ('water', 30622), ('garlic', 27853), ('onion', 26057), ('juice', 22202)]
Top 5 bigrams: [['ts salt', 32936], ['salt ts', 12378], ['to taste', 12196], ['olive oil', 12155], ['finely chopped', 11261]], top 5 trigrams: [['ts salt ts', 10473], ['all purpose flour', 6940], ['ts baking powder', 6209], ['pepper to taste', 5745], ['tb olive oil', 5733]]
==

In [ ]:
get_ds_statistics(dev_df_orig)

In [ ]:
get_ds_statistics(test_df_orig)

### Data Preprocessing

In [ ]:
def replace(df, patterns, replacements, columns, regex=False):
    # patterns: List[str]
    # replacements: List[str]
    # columns: List[str]
    if type(replacements) == str:
        replacements = [replacements] * len(patterns)
    if type(columns) == str:
        columns = [columns] * len(patterns)

    for pat, rep, col in zip(patterns, replacements, columns):
        df[col] = df[col].str.replace(pat, rep, regex=regex)

def add_tags(df, add_intermediate_tag=False):
    assert 'Ingredients' in df.columns and 'Recipe' in df.columns

    replace(df, ['\t'], ' <INGREDIENT> ' if add_intermediate_tag else ' ', 'Ingredients')
    df.Ingredients = '<INGREDIENT_START> ' + df.Ingredients + ' <INGREDIENT_END>'

    replace(df, ['.', ';'], ' <RECIPE_STEP> ' if add_intermediate_tag else ' ', 'Recipe')
    df.Recipe = '<RECIPE_START> ' + df.Recipe + ' <RECIPE_END>'

def preprocess_data(orig_df, max_ingr_len=150, max_recipe_len=600, min_recipe_len=5, min_ingredients=1,
                    add_intermediate_tag=False):
    df = orig_df.copy() # ensure original data is not mutated (mostly for verification purposes)

    ## drop NA
    df = df.dropna()

    ## keep only rows with all lowercase (Recipe column is all lowercase already)
    df = df[df.Ingredients.str.islower()]

    ## replace brackets with space
    replace(df, ['[()]'], ' ', ['Ingredients', 'Recipe'], regex=True)

    ## add spaces around non-words (exclude whitespace, apostrophe, period (treated separately below))
    replace(df, ["([^0-9a-zA-Z.'\"/ ])"]*2, r" \1 ", ['Ingredients', 'Recipe'], regex=True)
    # add spaces around periods (excluding decimal places)
    replace(df, [r"\.(?!\d)"]*2, r" . ", ['Ingredients', 'Recipe'], regex=True)
    # add spaces around word/word
    replace(df, [r"([^0-9])\/([^0-9])"]*2, r"\1 / \2", ['Ingredients', 'Recipe'], regex=True)

    ## add tags for ingredients and recipes
    add_tags(df, add_intermediate_tag=add_intermediate_tag)

    ## replace >1 whitespace with a single space
    replace(df, ['[ ]{2,}']*2, " ", ['Ingredients', 'Recipe'], regex=True)

    ## remove leading and trailing whitespace
    df.Ingredients = df.Ingredients.str.strip()
    df.Recipe = df.Recipe.str.strip()

    if add_intermediate_tag:
        ## remove consecutive tags, for ex. <INGREDIENT>[0 or more whitespace]<INGREDIENT>
        replace(df, ["<INGREDIENT>[ \t\n]*([ \t\n]*<INGREDIENT>)+", "<RECIPE_STEP>[ \t\n]*([ \t\n]*<RECIPE_STEP>)+"], 
        ["<INGREDIENT>", "<RECIPE_STEP>"], ["Ingredients", "Recipe"], regex=True)

    ## filter out recipes and ingredients above/below limit
    recipe_lens = df.Recipe.apply(lambda r: len(r.split()))
    df = df[(recipe_lens > min_recipe_len) & (recipe_lens < max_recipe_len)]
    df = df[df.Ingredients.apply(lambda i: len(i.split())) < max_ingr_len]

    if add_intermediate_tag:
        ## filter out those with <1 ingredients
        df = df[df.Ingredients.str.count('<INGREDIENT>') >= min_ingredients]

    df = df.reset_index(drop=True)

    print(f"Number of data samples before preprocessing: {len(orig_df)}\n"
          f"Number of data samples after preprocessing: {len(df)} ({len(df) * 100/len(orig_df):.3f}%)")

    return df

In [11]:
train_df = preprocess_data(train_df_orig, max_ingr_len=MAX_INGR_LEN, max_recipe_len=MAX_RECIPE_LEN, add_intermediate_tag=add_intermediate_tag)

Number of data samples before preprocessing: 101340
Number of data samples after preprocessing: 100637 (99.306%)


In [12]:
dev_df = preprocess_data(dev_df_orig, max_ingr_len=MAX_INGR_LEN, max_recipe_len=MAX_RECIPE_LEN, add_intermediate_tag=add_intermediate_tag)

Number of data samples before preprocessing: 797
Number of data samples after preprocessing: 793 (99.498%)


In [13]:
test_df = preprocess_data(test_df_orig, max_ingr_len=MAX_INGR_LEN, max_recipe_len=MAX_RECIPE_LEN, add_intermediate_tag=add_intermediate_tag)

Number of data samples before preprocessing: 778
Number of data samples after preprocessing: 774 (99.486%)


### Defining vocabulary

In [14]:
class Vocabulary:
    def __init__(self, add_intermediate_tag=False):
        """Vocabulary class which can convert a valid word to unique index and converting the index back to word."""
        special_tags = dict(SPECIAL_TAGS)
        if not add_intermediate_tag:
            special_tags.pop(ING)
            special_tags.pop(REC)
        ## initialize
        self._word2index = special_tags
        self.word2count = {k: 0 for k in special_tags.keys()}
        self.index2word = {v:k for k,v in special_tags.items()}
        self.n_unique_words = len(self.index2word) # total number of words in the dictionary.

    def __len__(self):
        return len(self._word2index)

    def add_word(self, word):
        if word not in self._word2index:
            self._word2index[word] = self.n_unique_words
            self.index2word[self.n_unique_words] = word
            self.n_unique_words += 1
            self.word2count[word] = 1
        else:
            self.word2count[word] += 1

    def word_exist_in_vocab(self, word):
        return word in self._word2index

    def word2index(self, word):
        if word not in self._word2index:
            return self._word2index[UNKNOWN_WORD]
        return self._word2index[word]

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_padding(self):
        # NOTE: should be called after finished with building vocab
        self.add_word(PAD_WORD)

    def add_unknown(self):
        self.add_word(UNKNOWN_WORD)

    def populate(self, df):
        for rowid in tqdm(range(len(df))):
            df_row = df.iloc[rowid]
            for i in range(2):
                self.add_sentence(df_row.iloc[i])
        self.add_unknown() # unknown word is for words in the dev/test not present in train
        self.add_padding() # padding should be last in the vocabulary (for convenience in decoder)

In [15]:
vocab = Vocabulary(add_intermediate_tag=add_intermediate_tag)
vocab.populate(train_df)
vocab.n_unique_words

100%|██████████| 100637/100637 [00:12<00:00, 8000.72it/s]


44315

In [16]:
dev_vocab = Vocabulary(add_intermediate_tag=add_intermediate_tag)
dev_vocab.populate(dev_df)
dev_vocab.n_unique_words

100%|██████████| 793/793 [00:00<00:00, 5427.67it/s]


4787

In [17]:
test_vocab = Vocabulary(add_intermediate_tag=add_intermediate_tag)
test_vocab.populate(test_df)
test_vocab.n_unique_words

100%|██████████| 774/774 [00:00<00:00, 7195.51it/s]


4550

### Creating Custom `torch.utils.data.Dataset`

In [ ]:
class RecipeDataset(Dataset):
    def __init__(self, df, vocab, train=True):
        """
        Args:
            df (pd.DataFrame): dataframe with two columns: "Ingredients" and "Recipe"
            vocab (Vocabulary): to convert word2index
        """
        super().__init__()
        self.ingredient_recipe_df = df
        self.vocab = vocab
        self.train = train

    def __len__(self):
        return len(self.ingredient_recipe_df)
    
    def __getitem__(self, index):
        row = self.ingredient_recipe_df.iloc[index]
        ingredient_tens = torch.tensor([self.vocab.word2index(w) for w in row.Ingredients.split(" ")],
                                       dtype=torch.long, device=DEVICE)
        if self.train:
            recipe_tens = torch.tensor([self.vocab.word2index(w) for w in row.Recipe.split(" ")],
                                        dtype=torch.long, device=DEVICE)
        else:
            recipe_tens = row.Recipe.split(" ") # List[str]
        return (ingredient_tens, recipe_tens)
    
# inspired by https://suzyahyah.github.io/pytorch/2019/07/01/DataLoader-Pad-Pack-Sequence.html
def pad_collate(vocab, train=True):

    def _pad_collate(batch):
        # print(len(batch))
        # print(batch[0])
        # ingredients: tuple of len batch_size with Tensor elements containing all ingredients in batch
        # recipes: tuple of len batch_size with Tensor elements containing all recipes in batch
        #           (or in eval:) tuple of len batch_size with elements List[str]
        # print(batch)
        ingredients, recipes = zip(*batch)
        # print(ingredients)
        # print(recipes)
        ingr_lens = torch.tensor([len(x) for x in ingredients], dtype=torch.long, device=DEVICE)
        ingredients_padded = pad_sequence(ingredients, batch_first=True, padding_value=vocab.word2index(PAD_WORD))

        if train:
            recipe_lens = torch.tensor([len(r) for r in recipes], dtype=torch.long, device=DEVICE)
            recipes_padded = pad_sequence(recipes, batch_first=True, padding_value=vocab.word2index(PAD_WORD))
        else:
            recipe_lens = None
            recipes_padded = list(recipes)

        return ingredients_padded, recipes_padded, ingr_lens, recipe_lens
    
    return _pad_collate

def pack(x_embed, x_lens):
    # convert tensor with padding to a PackedSequence, this allows rnns to ignore paddings
    return pack_padded_sequence(x_embed, x_lens.cpu().int(), batch_first=True, enforce_sorted=False)

def unpack(out_packed, padding_val):
    out_padded, out_lens = pad_packed_sequence(out_packed, batch_first=True, padding_value=padding_val)
    return out_padded, out_lens

In [ ]:
train_ds = RecipeDataset(train_df, vocab)
# used for getting validation loss
dev_ds_val_loss = RecipeDataset(dev_df, vocab, train=True) 
# used for getting validation BLEU, and other metrics
dev_ds_val_met = RecipeDataset(dev_df, vocab, train=False) 
test_ds = RecipeDataset(test_df, vocab, train=False)

## Defining Training, Validation, and Testing Functions

### Testing Functions

In [26]:
def get_ingredients_regex(ingredients_lst):
    return r'\b(?:' + '|'.join(re.escape(i) for i in ingredients_lst) + r')\b'

def get_invalid_ingredients_regex(ingredients_lst):
    # return r'(^|\.\s|,\s)(\b(?:' + '|'.join(re.escape(ingredient) for ingredient in ingredients_lst) + r')\b)'
    return r'(^|\.\s)(\b(?:' + '|'.join(re.escape(ingredient) for ingredient in ingredients_lst) + r')\b)'

def get_all_ingredients(fpath, reverse_sort=True):
    # get list of valid ingredients in the dataset from RecipeWithPlans
    with open(fpath, "r") as f:
        all_ings = json.load(f)
    if reverse_sort:
        # sort by longest to shortest ingredients
        all_ings = sorted(all_ings, key=lambda i: len(i.split()), reverse=True)
    return all_ings

def find_ingredients_in_text(txt, regex, enforce_unique=True):
    res = re.findall(regex, txt)
    if enforce_unique:
        res = set(res)
    return res

def calc_bleu(gt_recipes, gen_recipes, split_gt=False, split_gen=False):
    """Calculate corpus BLEU-4 score.

    Args:
        gt_recipes (List): len N
        gen_recipes (List[List] or List): 
    """
    gt_recipes_lst2 = [[word_tokenize(gt)] if split_gt else [gt] for gt in gt_recipes]
    if split_gen:
        gen_recipes = [word_tokenize(r) for r in gen_recipes]
    return corpus_bleu(gt_recipes_lst2, gen_recipes)

def calc_meteor(gt_recipes, gen_recipes, split_gt=False, split_gen=False):
    if split_gt:
        gt_recipes_lst2 = [word_tokenize(gt) for gt in gt_recipes]
    else:
        gt_recipes_lst2 = gt_recipes

    meteor_score = 0
    for i in tqdm(range(len(gen_recipes))):
        generated_recipe = word_tokenize(gen_recipes[i]) if split_gen else gen_recipes[i]
        gt_recipes_i = gt_recipes_lst2[i]
        meteor_score += meteor([gt_recipes_i], generated_recipe)
    return meteor_score / len(gen_recipes)

def eval_decoder_iter(decoder, decoder_hidden, decoder_cell, encoder_houts,
                      ingredients, max_recipe_len, vocab, decoder_mode="basic"):
    assert decoder_mode in ["basic", "attention"]
    
    N = ingredients.size(0)
    all_decoder_outs = [[REC_START] for _ in range(N)] # stores the decoder outputs for each batch sample

    valid = torch.ones([N], device=DEVICE).bool() # Tensor[N] 
    decoder_input = torch.full([N], SPECIAL_TAGS[REC_START], dtype=torch.long, device=DEVICE)
    
    for _ in range(max_recipe_len-1): # generations are bounded by max length (-1 because of EOS)
        decoder_hidden_i = decoder_hidden[:, valid] # [1, N_valid, H]
        decoder_cell_i = decoder_cell[:, valid]

        if decoder_mode == "basic":
            # decoder_out: log probabilities over vocab; [N_valid, |Vocab|-1]
            # decoder_hfinal: final hidden state; [num_layers=1, N_valid, H]
            decoder_out, decoder_hidden_i, decoder_cell_i = decoder(decoder_input, decoder_hidden_i, decoder_cell_i)
        elif decoder_mode == "attention":
            encoder_houts_i = encoder_houts[valid] # [N_valid, L_i, H]
            decoder_out, decoder_hidden_i, decoder_cell_i, attn_weights_i = decoder(
                decoder_input, decoder_hidden_i, decoder_cell_i, encoder_houts_i, ingredients[valid])
            
        # decoder_tok_preds: token with highest log probability
        decoder_topk_preds = decoder_out.topk(1)[1].reshape(-1) # [N_valid]

        ## store generated output
        for dec_idx, valid_n in zip(range(len(decoder_topk_preds)), valid.nonzero()):
            valid_idx = valid_n.item()
            all_decoder_outs[valid_idx].append(
                vocab.index2word[decoder_topk_preds[dec_idx].item()]) # str

        ## check for end of recipe
        not_eor= decoder_topk_preds != SPECIAL_TAGS[REC_END] # [N_valid]
        # update valid
        valid_temp = valid.clone() # to avoid single mem location error
        valid_temp[valid] = not_eor
        valid = valid_temp
        del valid_temp
        # valid = torch.logical_and(valid, not_eor)

        # update decoder input for next iteration
        decoder_input = decoder_topk_preds[not_eor] # [N_valid_next]

        # update only valid decoder_hidden
        decoder_hidden[:, valid] = decoder_hidden_i[:, not_eor]
        decoder_cell[:, valid] = decoder_cell_i[:, not_eor]

        if valid.sum() < 1:
            break
    else: # if did not break meaning 1 or more exceeded max generation limit
        # forcably insert recipe stop tok at the end
        for valid_n in valid.nonzero():
            valid_idx = valid_n.item()
            all_decoder_outs[valid_idx].append(REC_END)

    return all_decoder_outs

def get_predictions_iter(ingredients, ing_lens, encoder, decoder, vocab, max_recipe_len=600, 
                         decoder_mode="basic"):
    """Get predictions from trained model for a single iteration. Processes batched data.
    NOTE: ensure that this function is wrapped in `with torch.no_grad():`

    Args:
        ingredients (torch.Tensor): padded ingredients tensor in idx form; 
                                    shape [N, L_i], where L_i = max ingredients length in batch
        ing_lens (torch.Tensor): unpadded length of ingredients; shape [N]
        rec_lens (torch.Tensor): unpadded length of recipes; shape [N]
        encoder (EncoderRNN): encoder RNN module
        decoder (DecoderRNN): decoder RNN module
    """
    assert encoder.training is False and decoder.training is False

    N = ingredients.size(0)

    ## feed ingredients through encoder
    # enc_out: padded encoder output tensor with shape [N, L, H]
    # enc_out_lens: unpadded sequence lengths; tensor with shape [N]
    # enc_h_final: final hidden state: [num_layers=1, N, H]
    # enc_c_final: final cell state: [num_layers=1, N, H]
    enc_out, enc_out_lens, enc_h_final, enc_c_final = encoder(ingredients, ing_lens)
    
    # initialize decoder hidden state as final encoder hidden state
    decoder_hidden = enc_h_final
    decoder_cell = enc_c_final

    # List[List[str]]
    
    all_decoder_outs = eval_decoder_iter(decoder, decoder_hidden, decoder_cell,
                                         enc_out, ingredients, max_recipe_len,
                                         vocab, decoder_mode=decoder_mode)

    return all_decoder_outs

def eval(encoder, decoder, dataset, vocab, batch_size=4, max_recipe_len=600, decoder_mode="basic"):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=pad_collate(vocab, train=False))

    all_decoder_outs = [] # (List[List[str]]): List of len `N`, each element is the generated sequence for that sample
    all_gt_recipes = [] # (List[List[str]])
    all_gt_ingredients = []

    encoder.eval()
    decoder.eval()

    with torch.no_grad():
        for ingredients, recipes, ing_lens, _ in tqdm(dataloader):
            # ingredients: Tensor[N, L_i] padded ingredients
            # recipes (List[List[str]]): list of len N, each element is a
            #                               list of len `|gt_i|`, which is the length of the i-th ground-truth sequence

            # dec_outs (List[List[str]]): list of len `batch_size`, each element is a 
            #                               list of len `gen_size`, which is the size of the generated sequence, and each element is a 
            #                                   str representing a single word in the generated recipe
            dec_outs = get_predictions_iter(ingredients, ing_lens, encoder, decoder, vocab, 
                                            max_recipe_len=max_recipe_len, decoder_mode=decoder_mode)
            
            all_decoder_outs += dec_outs
            all_gt_recipes += recipes
            all_gt_ingredients += ingredients.tolist()

    return all_decoder_outs, all_gt_recipes, all_gt_ingredients

### Validation Functions

In [ ]:
def train_decoder_iter(decoder, decoder_hidden, decoder_cell, encoder_houts, 
                       ingredients, recipes, padded_rec_len, rec_lens,
                       pad_word_idx, decoder_mode="basic"):
    assert decoder_mode in ["basic", "attention"]

    all_decoder_outs = [] # List of [N, |Vocab|-1]
    all_gt = [] # List of [N]
    ## NOTE: recipes already contain start token no need to add manually
    # encoder_houts_i = encoder_houts # [N, L_i, H]
    for di in range(padded_rec_len-1):
        # get batches which have valid (non-padding and non ending) tokens as input
        valid = (rec_lens - 1) > di
        decoder_input_i = recipes[valid, di] # [N_valid]
        decoder_hidden_i = decoder_hidden[:,valid] # [n_layers=1, N_valid, H]
        decoder_cell_i = decoder_cell[:, valid] # [n_layers=1, N_valid, H]

        if decoder_mode == "basic":
            # decoder_out: log probabilities over vocab; [N_valid, |Vocab|-1]
            # decoder_hfinal: final hidden state; [num_layers=1, N_valid, H]
            decoder_out, decoder_hidden_i, decoder_cell_i = decoder(
                decoder_input_i, decoder_hidden_i, decoder_cell_i)
            attn_weights = None
        elif decoder_mode == "attention":
            encoder_houts_i = encoder_houts[valid] # [N_valid, L_i, H]
            decoder_out, decoder_hidden_i, decoder_cell_i, attn_weights_i = decoder(
                decoder_input_i, decoder_hidden_i, decoder_cell_i, encoder_houts_i, ingredients[valid])

        all_decoder_outs.append(decoder_out)

        # because we ensured that input cannot be end token, there is a guaranteed non-padding token
        # for each valid batch sample
        gt_i = recipes[valid, di+1] # [N_valid]
        assert (gt_i != pad_word_idx).all(), f"gt_i should not have padding but got: {gt_i}"
        all_gt.append(gt_i)

        # update only valid decoder_hidden and decoder_cell
        decoder_hidden[:, valid] = decoder_hidden_i
        decoder_cell[:, valid] = decoder_cell_i

    return all_decoder_outs, all_gt

def get_validation_loss_iter(encoder, decoder, ingredients, recipes, ing_lens, rec_lens, vocab, criterion,
                             decoder_mode="basic"):
    assert not encoder.training and not decoder.training
    padded_rec_len = recipes.size(1) # L_r

    ## feed ingredients through encoder
    # enc_out: padded encoder output tensor with shape [N, L, H]
    # enc_out_lens: unpadded sequence lengths; tensor with shape [N]
    # enc_h_final: final hidden state: [num_layers=1, N, H]
    # enc_c_final: final cell state: [num_layers=1, N, H]
    enc_out, enc_out_lens, enc_h_final, enc_c_final = encoder(ingredients, ing_lens)

    # initialize decoder hidden state and cell state as final encoder hidden and cell state
    decoder_hidden = enc_h_final
    decoder_cell = enc_c_final

    if TEACHER_FORCING_RATIO < 1:
        raise ValueError("Non-teacher forcing is not implemented")
    
    loss = 0

    all_decoder_outs, all_gt = train_decoder_iter(decoder, decoder_hidden, decoder_cell, enc_out, 
                                                  ingredients, recipes, padded_rec_len, rec_lens, 
                                                  vocab.word2index(PAD_WORD), decoder_mode=decoder_mode)
    
    all_decoder_outs = torch.cat(all_decoder_outs, dim=0)
    all_gt = torch.cat(all_gt, dim=0)

    # mean Negative Log Likelihood Loss
    loss = criterion(all_decoder_outs, all_gt)

    return loss.item()


def get_validation_loss(encoder, decoder, dataset, vocab, batch_size=4, decoder_mode="basic"):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=pad_collate(vocab))

    encoder.eval()
    decoder.eval()

    criterion = nn.NLLLoss()

    total_loss = 0

    with torch.no_grad():
        for idx, (ingredients, recipes, ing_lens, rec_lens) in tqdm(enumerate(dataloader)):
            loss = get_validation_loss_iter(encoder, decoder, ingredients, recipes, ing_lens, rec_lens,
                                            vocab, criterion, decoder_mode=decoder_mode)
            total_loss += loss

    return total_loss/len(dataloader)

### Training Function

In [ ]:
def save_model(encoder, decoder, identifier):
    """_summary_

    Args:
        encoder (_type_): _description_
        decoder (_type_): _description_
        identifier (str): suffix of the filename should not include "encoder" or "decoder" and does not inclue file extension, .pth
    """
    if not os.path.isdir("./saved_models"):
        os.makedirs("saved_models")
    
    encpath = os.path.join("saved_models", f"encoder_{identifier}.pth")
    decpath = os.path.join("saved_models", f"decoder_{identifier}.pth")

    torch.save(encoder.state_dict(), encpath)
    torch.save(decoder.state_dict(), decpath)

def load_model(encoder, decoder, identifier):
    """_summary_

    Args:
        encoder (_type_): _description_
        decoder (_type_): _description_
        identifier (str): suffix of the filename should not include "encoder" or "decoder" and does not inclue file extension, .pth
    """
    encpath = os.path.join("saved_models", f"encoder_{identifier}.pth")
    decpath = os.path.join("saved_models", f"decoder_{identifier}.pth")

    encoder.load_state_dict(torch.load(encpath))
    decoder.load_state_dict(torch.load(decpath))

def save_log(identifier, train_out, enc_optim, dec_optim, enc_sched, dec_sched):
    if not os.path.isdir("./logs"):
        os.makedirs("logs") 
    savepath = os.path.join("logs",f"log_{identifier}.txt") 
    with open(savepath, "w") as f:
        f.write(str(enc_optim) + "\n")
        f.write(str(dec_optim) + "\n")
        f.write(str(enc_sched) + "\n")
        f.write(str(dec_sched) + "\n")
        f.write("========================\n")
        f.write(train_out)

def train_iter(ingredients, recipes, ing_lens, rec_lens, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion,
               decoder_mode="basic", prevent_pretrained_grad_update=True, vocab=None # !remove later
               ):
    """Single training iteration. Processes batched data.

    Args:
        ingredients (torch.Tensor): padded ingredients tensor in idx form; 
                                    shape [N, L_i], where L_i = max ingredients length in batch
        recipes (torch.Tensor): padded recipes tensor in idx form;
                                shape [N, L_r], where L_r = max recipes length in batch
        ing_lens (torch.Tensor): unpadded length of ingredients; shape [N]
        rec_lens (torch.Tensor): unpadded length of recipes; shape [N]
        encoder (EncoderRNN): encoder RNN module
        decoder (DecoderRNN): decoder RNN module
        encoder_optimizer (torch.optim)
        decoder_optimizer (torch.optim)
        criterion (torch.nn.NLLLoss): loss function
    """

    ## reset gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    N = ingredients.size(0)
    padded_ing_len = ingredients.size(1) # L_i
    padded_rec_len = recipes.size(1) # L_r


    ## feed ingredients through encoder
    # enc_out: padded encoder output tensor with shape [N, L, H]
    # enc_out_lens: unpadded sequence lengths; tensor with shape [N]
    # enc_h_final: final hidden state: [num_layers=1, N, H]
    # enc_c_final: final cell state: [num_layers=1, N, H]
    enc_out, enc_out_lens, enc_h_final, enc_c_final = encoder(ingredients, ing_lens)

    # initialize decoder hidden state and cell state as final encoder hidden and cell state
    decoder_hidden = enc_h_final
    decoder_cell = enc_c_final

    if TEACHER_FORCING_RATIO < 1:
        raise ValueError("Non-teacher forcing is not implemented")
    
    loss = 0

    all_decoder_outs, all_gt = train_decoder_iter(decoder, decoder_hidden, decoder_cell, enc_out, 
                                                  ingredients, recipes, padded_rec_len, rec_lens, 
                                                  vocab.word2index(PAD_WORD), decoder_mode=decoder_mode)
    
    all_decoder_outs = torch.cat(all_decoder_outs, dim=0)
    all_gt = torch.cat(all_gt, dim=0)

    # mean Negative Log Likelihood Loss
    loss = criterion(all_decoder_outs, all_gt)

    ## backpropagation
    loss.backward()

    # print("=====BEFORE MASKING=====")
    # print("ENCODER EMBEDDING WEIGHT GRADS of shape:", encoder.embedding.weight.grad.shape)
    # print(encoder.embedding.weight.grad[:10, :10])

    # print("DECODER EMBEDDING WEIGHT GRADS of shape:", decoder.embedding.weight.grad.shape)
    # print(decoder.embedding.weight.grad[:10, :10])

    # print("=====AFTER MASKING=====")
    if prevent_pretrained_grad_update:
        encoder.update_embedding_grad(encoder.embedding.weight.grad)
        decoder.update_embedding_grad(decoder.embedding.weight.grad)
    # print("ENCODER EMBEDDING WEIGHT GRADS of shape:", encoder.embedding.weight.grad.shape)
    # print(encoder.embedding.weight.grad[:10, :10])

    # print("DECODER EMBEDDING WEIGHT GRADS of shape:", decoder.embedding.weight.grad.shape)
    # print(decoder.embedding.weight.grad[:10, :10])


    ## update params
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item()

def train(encoder, decoder, encoder_optimizer, decoder_optimizer, dataset, n_epochs, vocab,
          decoder_mode="basic", batch_size=4, enc_lr_scheduler=None, dec_lr_scheduler=None, 
          dev_ds_val_loss=None, dev_ds_val_met=None, identifier="", min_bleu_to_save=0.01,
          verbose=True, verbose_iter_interval=10):
    assert (enc_lr_scheduler is None and dec_lr_scheduler is None) or (enc_lr_scheduler is not None and dec_lr_scheduler is not None)
    
    evaluate = dev_ds_val_loss is not None and dev_ds_val_met is not None
    assert (not evaluate) or (evaluate and len(identifier) > 0)
    
    use_scheduler =  enc_lr_scheduler is not None and dec_lr_scheduler is not None
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=pad_collate(vocab))
    total_iters = len(dataloader)
    
    
    epoch_losses = torch.zeros(size=[n_epochs], dtype=torch.double, device=DEVICE, requires_grad=False)
    val_epoch_losses = torch.zeros(size=[n_epochs], dtype=torch.double, device=DEVICE, requires_grad=False)
    
    criterion = nn.NLLLoss()
    log = ""
    
    highest_bleu = 0

    for epoch in range(n_epochs):
        if verbose: print(f"Starting epoch {epoch+1}/{n_epochs}, "
                          f"enc lr scheduler: {enc_lr_scheduler.get_last_lr()}, dec lr scheduler: {dec_lr_scheduler.get_last_lr()}" \
                            if use_scheduler else "")
        epoch_loss = 0 # accumulate total loss during epoch
        print_epoch_loss = 0 # accumulate losses for printing

        start_epoch_time = time.time()
        for iter_idx, (ingredients, recipes, ing_lens, rec_lens) in enumerate(dataloader):
            if verbose and iter_idx > 0  and iter_idx % verbose_iter_interval == 0:
                msg = f"(Epoch {epoch}, iter {iter_idx}/{total_iters}) Average loss so far: {print_epoch_loss/verbose_iter_interval:.3f}"
                log += msg + "\n"
                print(msg)
                print_epoch_loss = 0
            loss = train_iter(ingredients, recipes, ing_lens, rec_lens, encoder, decoder, 
                                   encoder_optimizer, decoder_optimizer, criterion, 
                                   decoder_mode=decoder_mode,
                                   vocab=vocab # remove later
                                   )
            epoch_loss += loss
            print_epoch_loss += loss
        end_epoch_time = time.time()
        epoch_loss /= total_iters # get average epoch loss
        if verbose: 
            one_epoch_time_sec = end_epoch_time - start_epoch_time
            remaining_epochs = n_epochs - epoch - 1
            remaining_time = one_epoch_time_sec * remaining_epochs
            remaining_time_hours = remaining_time //3600
            remaining_time_mins = remaining_time % 3600 // 60
            msg = f"Average epoch loss: {epoch_loss:.3f}\n"\
                  f"This epoch took {one_epoch_time_sec / 60} mins. Time remaining: {remaining_time_hours} hrs {remaining_time_mins} mins."
            log += msg + "\n"
            print(msg)

        epoch_losses[epoch]=epoch_loss
        if use_scheduler:
            enc_lr_scheduler.step()
            dec_lr_scheduler.step()

        if evaluate:
            ## get validation loss
            val_loss = get_validation_loss(encoder, decoder, dev_ds_val_loss, vocab, batch_size=batch_size,
                                           decoder_mode=decoder_mode)
            val_epoch_losses[epoch] = val_loss
            
            if verbose:
                msg = f"validation loss: {val_loss}"
                log += msg + "\n"
                print(msg)

            ## get validation metrics
            all_decoder_outs, all_gt_recipes, _ = eval(encoder, decoder, dev_ds_val_met, vocab,
                                                    batch_size=batch_size, max_recipe_len=MAX_RECIPE_LEN,
                                                    decoder_mode=decoder_mode)
            bleu = calc_bleu(all_gt_recipes, all_decoder_outs)
            meteor = calc_meteor(all_gt_recipes, all_decoder_outs, split_gt=False)

            if verbose:
                msg = f"BLEU score: {bleu}, METEOR score: {meteor}"
                log += msg + "\n"
                print(msg)
            if bleu > highest_bleu:
                highest_bleu = bleu
                if bleu > min_bleu_to_save:
                    save_model(encoder, decoder, f"{identifier}_ep_{epoch}")
            
            # set back to train mode because in eval they are set to eval mode
            encoder.train()
            decoder.train()


    return epoch_losses, val_epoch_losses, log

# Implementation of Baseline 1

### Defining Base Encoder-Decoder Sequence-to-Sequence Model

In [ ]:
# NOTE: the following two functions are for Extension 1 but are placed here so that we can reuse
# NOTE: this encoder and decoder class

def create_pretrained_embeddings(pretrained_embedding_dict, input_size, embedding_size, vocab, verbose=True):
    """
    NOTE: input_size is typically len(vocab) although in decoder you don't need to generate
    unknown and pad.
    """
    emb = torch.randn([input_size, embedding_size])

    vocab_idxs_in_pretrained = []
    vocab_words = list(vocab._word2index.keys())
    for i in tqdm(range(input_size)):
        word = vocab_words[i]
        if word in pretrained_embedding_dict:
            vocab_idxs_in_pretrained.append(i)
            emb[i] = pretrained_embedding_dict[word]
    
    if verbose: 
        print(f"{len(vocab_idxs_in_pretrained)}/{input_size} ({len(vocab_idxs_in_pretrained)/input_size:.3f}%) words have pretrained embeddings")

    return emb, vocab_idxs_in_pretrained

def create_embeddings(input_size, embedding_size, pretrained_embedding_dict, vocab):
    if pretrained_embedding_dict is None:
        return nn.Embedding(input_size, embedding_size), None
    else:
        assert vocab is not None
        embeddings_w_pretrained, vocab_idxs_in_pretrained = create_pretrained_embeddings(
            pretrained_embedding_dict, input_size, embedding_size, vocab
        )
        # ! For now not freezing embeddings because ~40% don't exist in pretrained embeddings
        # plus allows for finetuning
        return nn.Embedding.from_pretrained(embeddings_w_pretrained, freeze=False), vocab_idxs_in_pretrained


class EncoderRNN(nn.Module):
    def __init__(self,
                 input_size,
                 embedding_size,
                 hidden_size,
                 padding_value,
                 pretrained_embedding_dict=None,
                 vocab=None,
                 num_lstm_layers=1
                 ):
        """Encoder LSTM to encode input sequence.

        input_size (int): size of vocabulary
        hidden_size (int): size of hidden dimension, referred to as H
        """
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding, self.pretrained_emb_idxs = create_embeddings(input_size, embedding_size, pretrained_embedding_dict, vocab)
        self.lstm = nn.LSTM(embedding_size, hidden_size, batch_first=True, num_layers=num_lstm_layers)
        self.padding_value = padding_value

    def update_embedding_grad(self, grads):
        if not hasattr(self, "pretrained_emb_idxs"):
            return
        
        grads[self.pretrained_emb_idxs] = 0 # zero-out gradients for pretrained embeddings

    def forward(self, ingredients, ing_lens):
        """Embed ingredients and feed through LSTM. 
        Batch process all words in sequence at once for efficiency rather than one word one batch at a time.

        Args:
            ingredients (torch.Tensor): padded ingredients of shape [N, L], where N=batch size and L=longest sequence length in batch
        """
        ## embed ingredients
        ingredients_embed = self.embedding(ingredients) # [N, L, E]

        ## pack padded ingredients tensor before feeding through LSTM (this allows the lstm to optimize operations, ignoring padding)
        ingredients_packed = pack(ingredients_embed, ing_lens)

        ## feed through LSTM
        # by default, initial hidden state and initial cell state are zeros
        # output: PackedSequence containing hidden state for each token in sequence
        # final hidden state: Tensor [num_layers=1, N, H] NOTE: this is the last non-padded hidden state for each input sequence
        # c_final: last cell state Tensor [num_layers=1, N, H]
        output, (h_final, c_final) = self.lstm(ingredients_packed)

        ## unpack PackedSequence to get back our padded tensor
        # output_padded: padded output tensor which masks out encoder outputs for padding to 0; shape [N, L, H] NOTE: output_padded[:, -1] != h_final because of padding
        # output_lens: unpadded sequence lengths; tensor of shape [N]
        output_padded, output_lens = unpack(output, padding_val=self.padding_value)

        return output_padded, output_lens, h_final, c_final
    
#! IMPORTANT: MAKE SURE DECODER'S OUTPUT SIZE IS VOCAB SIZE - 1
class DecoderRNN(nn.Module):
    def __init__(self,
                 embedding_size,
                 hidden_size,
                 output_size,
                 pretrained_embedding_dict=None,
                 vocab=None,
                 ):
        """Decoder to generate recipes based on encoder output (hidden state(s)).

        Args:
            hidden_size (int): size of hidden dimension
            output_size (int): size of target language vocabulary - 1 (doesn't need to encode padding), |Vocab| - 1
        """
        super().__init__()
        self.hidden_size = hidden_size

        self.embedding, self.pretrained_emb_idxs = create_embeddings(output_size, embedding_size, pretrained_embedding_dict, vocab)
        self.lstm = nn.LSTM(embedding_size, hidden_size, batch_first=False)
        self.nonlinear_activation = nn.Tanh()
        self.out_fc = nn.Linear(hidden_size, output_size)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def update_embedding_grad(self, grads):
        if not hasattr(self, "pretrained_emb_idxs"):
            return
        
        grads[self.pretrained_emb_idxs] = 0 # zero-out gradients for pretrained embeddings

    def forward(self, inp, hidden, cell):
        """Decode one word at a time. Batch processed.

        Args:
            inp (torch.Tensor): start token or previous generation (non teacher-forcing) or 
                                previous ground truth token (teacher-forcing);
                                shape [N]
            hidden (torch.Tensor): encoder/decoder last hidden state; shape [1, N, H]
            cell (torch.Tensor): encoder/decoder last hidden state; shape [1, N, H]
        """

        inp_embedded = self.embedding(inp)[None] # [L=1, N, E]

        ## apply non-linear activation
        inp_embedded = self.nonlinear_activation(inp_embedded)
        
        ## feed embedded input and hidden state through LSTM
        # out: output features; shape [L=1, N, H]
        # h_final: final updated hidden state; shape [num_layers=1, N, H]
        # c_final: last cell state Tensor [num_layers=1, N, H]
        out, (h_final, c_final) = self.lstm(inp_embedded, (hidden, cell))

        ## linear projection
        out = self.out_fc(out[0]) # [N, H] -> [N, |Vocab|]

        ## log softmax to get log probability distribution over vocabulary words
        out = self.logsoftmax(out) # [N, |Vocab|]

        return out, h_final, c_final

### Training Baseline 1

In [ ]:
embedding_size=300
encoder = EncoderRNN(vocab.n_unique_words, embedding_size=embedding_size, hidden_size=HIDDEN_SIZE, padding_value=vocab.word2index(PAD_WORD)).to(DEVICE)
# in the training script, decoder is always fed a non-end token and thus never needs to generate padding
# also it should never generate "<UNKNOWN>"
decoder = DecoderRNN(embedding_size=embedding_size,hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-1).to(DEVICE)

In [ ]:
initial_lr=1e-3
min_lr = 1e-5
n_epochs = 30
batch_size=128
encoder_optimizer = optim.Adam(encoder.parameters(), lr=initial_lr)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=initial_lr)
enc_scheduler = CosineAnnealingLR(encoder_optimizer, T_max=n_epochs, eta_min=min_lr)
dec_scheduler = CosineAnnealingLR(decoder_optimizer, T_max=n_epochs, eta_min=min_lr)
# enc_scheduler = MultiStepLR(encoder_optimizer, milestones=[15], gamma=0.1)
# dec_scheduler = MultiStepLR(decoder_optimizer, milestones=[15], gamma=0.1)
identifier="adam_without_intermediate_tags_with_val_wd0_lr1e-3"
epoch_losses, val_epoch_losses, log = train(encoder, decoder, encoder_optimizer, decoder_optimizer, train_ds, 
                     n_epochs=n_epochs, vocab=vocab, decoder_mode="basic", batch_size=batch_size, 
                     enc_lr_scheduler=enc_scheduler, dec_lr_scheduler=dec_scheduler, 
                     dev_ds_val_loss = dev_ds_val_loss, dev_ds_val_met=dev_ds_val_met, identifier=identifier,
                     verbose_iter_interval=50)

save_log(identifier, log, encoder_optimizer, decoder_optimizer, enc_scheduler, dec_scheduler)

Starting epoch 1/30, enc lr scheduler: [0.001], dec lr scheduler: [0.001]
(Epoch 0, iter 50/787) Average loss so far: 7.144
(Epoch 0, iter 100/787) Average loss so far: 5.967
(Epoch 0, iter 150/787) Average loss so far: 5.730
(Epoch 0, iter 200/787) Average loss so far: 5.470
(Epoch 0, iter 250/787) Average loss so far: 5.212
(Epoch 0, iter 300/787) Average loss so far: 5.019
(Epoch 0, iter 350/787) Average loss so far: 4.865
(Epoch 0, iter 400/787) Average loss so far: 4.737
(Epoch 0, iter 450/787) Average loss so far: 4.632
(Epoch 0, iter 500/787) Average loss so far: 4.540
(Epoch 0, iter 550/787) Average loss so far: 4.449
(Epoch 0, iter 600/787) Average loss so far: 4.382
(Epoch 0, iter 650/787) Average loss so far: 4.317
(Epoch 0, iter 700/787) Average loss so far: 4.246
(Epoch 0, iter 750/787) Average loss so far: 4.207
Average epoch loss: 4.956
This epoch took 5.9502798040707905 mins. Time remaining: 2.0 hrs 52.0 mins.


7it [00:00,  8.22it/s]


validation loss: 4.188371658325195


100%|██████████| 793/793 [00:09<00:00, 87.30it/s] 


BLEU score: 0.0040273883646875595, METEOR score: 0.09091902403865486
Starting epoch 2/30, enc lr scheduler: [0.0009972883382072953], dec lr scheduler: [0.0009972883382072953]
(Epoch 1, iter 50/787) Average loss so far: 4.108
(Epoch 1, iter 100/787) Average loss so far: 4.071
(Epoch 1, iter 150/787) Average loss so far: 4.044
(Epoch 1, iter 200/787) Average loss so far: 4.004
(Epoch 1, iter 250/787) Average loss so far: 3.953
(Epoch 1, iter 300/787) Average loss so far: 3.925
(Epoch 1, iter 350/787) Average loss so far: 3.904
(Epoch 1, iter 400/787) Average loss so far: 3.880
(Epoch 1, iter 450/787) Average loss so far: 3.846
(Epoch 1, iter 500/787) Average loss so far: 3.816
(Epoch 1, iter 550/787) Average loss so far: 3.792
(Epoch 1, iter 600/787) Average loss so far: 3.775
(Epoch 1, iter 650/787) Average loss so far: 3.753
(Epoch 1, iter 700/787) Average loss so far: 3.747
(Epoch 1, iter 750/787) Average loss so far: 3.701
Average epoch loss: 3.878
This epoch took 5.9652024269104 min

...

BLEU score: 0.04576639965453813, METEOR score: 0.19814575935905007
Starting epoch 30/30, enc lr scheduler: [1.2711661792704668e-05], dec lr scheduler: [1.2711661792704668e-05]
(Epoch 29, iter 50/787) Average loss so far: 2.674
(Epoch 29, iter 100/787) Average loss so far: 2.674
(Epoch 29, iter 150/787) Average loss so far: 2.675
(Epoch 29, iter 200/787) Average loss so far: 2.647
(Epoch 29, iter 250/787) Average loss so far: 2.673
(Epoch 29, iter 300/787) Average loss so far: 2.653
(Epoch 29, iter 350/787) Average loss so far: 2.662
(Epoch 29, iter 400/787) Average loss so far: 2.661
(Epoch 29, iter 450/787) Average loss so far: 2.664
(Epoch 29, iter 500/787) Average loss so far: 2.695
(Epoch 29, iter 550/787) Average loss so far: 2.670
(Epoch 29, iter 600/787) Average loss so far: 2.660
(Epoch 29, iter 650/787) Average loss so far: 2.679
(Epoch 29, iter 700/787) Average loss so far: 2.666
(Epoch 29, iter 750/787) Average loss so far: 2.656
Average epoch loss: 2.668
This epoch took 5.9

7it [00:00,  8.36it/s]


validation loss: 3.191310610089983


100%|██████████| 793/793 [00:01<00:00, 697.13it/s]

BLEU score: 0.045667970073548995, METEOR score: 0.19664536921554326


# Implementation of Baseline 2

Note that the Encoder is reused from Baseline 1

In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, embedding_size, hidden_size, output_size, padding_val,
                 dropout=0.1, global_max_ing_len=MAX_INGR_LEN,
                 pretrained_embedding_dict=None,
                 vocab=None, num_lstm_layers=1,
                 ):
        super().__init__()
        self.hidden_size = hidden_size
        self.padding_val = padding_val
        self.global_max_ing_len = global_max_ing_len

        self.embedding, self.pretrained_emb_idxs = create_embeddings(
            output_size, embedding_size, pretrained_embedding_dict, vocab)
        self.attn = nn.Linear(hidden_size + embedding_size, global_max_ing_len)
        self.attn_combine = nn.Linear(hidden_size + embedding_size, hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=False, num_layers=num_lstm_layers)
        self.nonlinear_activation = nn.ReLU() # TODO: TRY TANH
        self.out_fc = nn.Linear(hidden_size, output_size)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def update_embedding_grad(self, grads):
        if not hasattr(self, "pretrained_emb_idxs"):
            return
        
        grads[self.pretrained_emb_idxs] = 0 # zero-out gradients for pretrained embeddings

    def mask_attn_weights(self, ingredients, attn_weights):
        """
        ingredients: [N, L_i]
        """
        # ing_len = ingredients.size(1)
        # pad ingredients on the right with `padding_val` to maximum len
        # [N, max_len]
        # padd_maxlen_ingredients = F.pad(
        #     ingredients, (0, self.global_max_ing_len-ing_len), value=0)
        # [N, L_i] where 1 is masked value, 0 is valid value
        attn_mask = ingredients == self.padding_val
        assert list(attn_weights.shape) == list(attn_mask.shape)
        attn_weights[attn_mask] = -torch.inf # set as -inf because when softmax-ed will turn to 0
        return attn_weights

    def forward(self, inp, hidden, cell, encoder_houts, ingredients):
        """
        Args:
            inp (torch.Tensor): start token or previous generation (non teacher-forcing) or 
                                previous ground truth token (teacher-forcing);
                                shape [N]
            hidden (torch.Tensor): encoder last hidden state; shape [1, N, H]
            cell (torch.Tensor): encoder last cell state; shape [1, N, H]
            encoder_houts (torch.Tensor): encoder all hidden states for all elements in sequence;
                                          padded tensor [N, L_i, H], where L_i is the max sequence len
        """
        L_i = encoder_houts.size(1) # max seq len in this batch
        ## embed token input
        inp_embedded = self.embedding(inp) # [N, E]

        inp_embedded = self.dropout(inp_embedded)

        attn_weights = self.attn(
            torch.cat((inp_embedded, hidden[-1]), dim=1) # [N, E+H]
        )[:, :L_i] # [N, max_ing_len] -> [N, L_i]
        # [N, L_i]
        attn_weights = F.softmax(
            self.mask_attn_weights(ingredients, attn_weights),
            dim=-1)
        # [N, 1, L_i] bmm [N, L_i, H] = [N, 1, H] -> [N, H]
        attn_res = torch.bmm(attn_weights[:, None], encoder_houts)[:, 0]

        # [N, E+H]
        output = torch.cat((inp_embedded, attn_res), dim=-1)
        # [N, E+H] -> [N, H] -> [L=1, N, H]
        output = self.attn_combine(output)[None]
        output = self.nonlinear_activation(output)

        ## feed embedded input and hidden state through LSTM
        # out: output features; shape [L=1, N, H]
        # h_final: final updated hidden state; shape [num_layers=1, N, H]
        # c_final: last cell state Tensor [num_layers=1, N, H]
        out, (h_final, c_final) = self.lstm(output, (hidden, cell))

        out = self.out_fc(out) # [L=1, N, H] -> [L=1, N, |Vocab|]

        ## log softmax to get log probability distribution over vocabulary words
        out = self.logsoftmax(out[0])

        return out, h_final, c_final, attn_weights

In [ ]:
reset_rng()
embedding_size=300
encoder_attn = EncoderRNN(vocab.n_unique_words, embedding_size=embedding_size, hidden_size=HIDDEN_SIZE, padding_value=vocab.word2index(PAD_WORD)).to(DEVICE)
# in the training script, decoder is always fed a non-end token and thus never needs to generate padding
# also it should never generate "<UNKNOWN>"
# decoder = DecoderRNN(embedding_size=embedding_size,hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-2).to(DEVICE)
decoder_attn = AttnDecoderRNN(embedding_size, hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-1, padding_val=vocab.word2index(PAD_WORD), 
                              dropout=DROPOUT).to(DEVICE)

In [ ]:
initial_lr=1e-3
min_lr = 1e-5
n_epochs = 30
batch_size=128
encoder_attn_optimizer = optim.Adam(encoder_attn.parameters(), lr=initial_lr)
decoder_attn_optimizer = optim.Adam(decoder_attn.parameters(), lr=initial_lr)
# enc_attn_scheduler = CosineAnnealingLR(encoder_attn_optimizer, T_max=n_epochs, eta_min=min_lr)
# dec_attn_scheduler = CosineAnnealingLR(decoder_attn_optimizer, T_max=n_epochs, eta_min=min_lr)
enc_attn_scheduler = CosineAnnealingLR(encoder_attn_optimizer, T_max=n_epochs, eta_min=min_lr)
dec_attn_scheduler = CosineAnnealingLR(decoder_attn_optimizer, T_max=n_epochs, eta_min=min_lr)
identifier="attn_adam_without_intermediate_tags_wd0_lr1e-3"

attn_epoch_losses, attn_val_epoch_losses, attn_log = train(
    encoder_attn, decoder_attn, encoder_attn_optimizer, decoder_attn_optimizer, train_ds, 
    n_epochs=n_epochs, vocab=vocab, decoder_mode="attention", batch_size=batch_size, 
    enc_lr_scheduler=enc_attn_scheduler, dec_lr_scheduler=dec_attn_scheduler, 
    dev_ds_val_loss = dev_ds_val_loss, dev_ds_val_met=dev_ds_val_met, identifier=identifier,
    verbose_iter_interval=50)

save_log(identifier, attn_log, encoder_attn_optimizer, decoder_attn_optimizer, 
         enc_attn_scheduler, dec_attn_scheduler)

Starting epoch 1/30, enc lr scheduler: [0.001], dec lr scheduler: [0.001]
(Epoch 0, iter 50/787) Average loss so far: 6.972
(Epoch 0, iter 100/787) Average loss so far: 6.023
(Epoch 0, iter 150/787) Average loss so far: 5.968
(Epoch 0, iter 200/787) Average loss so far: 5.927
(Epoch 0, iter 250/787) Average loss so far: 5.877
(Epoch 0, iter 300/787) Average loss so far: 5.818
(Epoch 0, iter 350/787) Average loss so far: 5.785
(Epoch 0, iter 400/787) Average loss so far: 5.725
(Epoch 0, iter 450/787) Average loss so far: 5.683
(Epoch 0, iter 500/787) Average loss so far: 5.621
(Epoch 0, iter 550/787) Average loss so far: 5.551
(Epoch 0, iter 600/787) Average loss so far: 5.440
(Epoch 0, iter 650/787) Average loss so far: 5.360
(Epoch 0, iter 700/787) Average loss so far: 5.215
(Epoch 0, iter 750/787) Average loss so far: 5.093
Average epoch loss: 5.701
This epoch took 8.194433637460072 mins. Time remaining: 3.0 hrs 57.0 mins.


7it [00:01,  5.45it/s]


validation loss: 4.93167952128819


100%|██████████| 793/793 [00:08<00:00, 95.55it/s] 


BLEU score: 0.002800126985071343, METEOR score: 0.08576382563509898
Starting epoch 2/30, enc lr scheduler: [0.0009972883382072953], dec lr scheduler: [0.0009972883382072953]
(Epoch 1, iter 50/787) Average loss so far: 4.863
(Epoch 1, iter 100/787) Average loss so far: 4.706
(Epoch 1, iter 150/787) Average loss so far: 4.641
(Epoch 1, iter 200/787) Average loss so far: 4.523
(Epoch 1, iter 250/787) Average loss so far: 4.437
(Epoch 1, iter 300/787) Average loss so far: 4.366
(Epoch 1, iter 350/787) Average loss so far: 4.295
(Epoch 1, iter 400/787) Average loss so far: 4.248
(Epoch 1, iter 450/787) Average loss so far: 4.186
(Epoch 1, iter 500/787) Average loss so far: 4.133
(Epoch 1, iter 550/787) Average loss so far: 4.082
(Epoch 1, iter 600/787) Average loss so far: 4.041
(Epoch 1, iter 650/787) Average loss so far: 4.004
(Epoch 1, iter 700/787) Average loss so far: 3.956
(Epoch 1, iter 750/787) Average loss so far: 3.932
Average epoch loss: 4.277
This epoch took 8.073306282361349 mi

...

BLEU score: 0.05513984026373013, METEOR score: 0.2094436932678814
Starting epoch 30/30, enc lr scheduler: [1.2711661792704668e-05], dec lr scheduler: [1.2711661792704668e-05]
(Epoch 29, iter 50/787) Average loss so far: 2.732
(Epoch 29, iter 100/787) Average loss so far: 2.722
(Epoch 29, iter 150/787) Average loss so far: 2.715
(Epoch 29, iter 200/787) Average loss so far: 2.739
(Epoch 29, iter 250/787) Average loss so far: 2.727
(Epoch 29, iter 300/787) Average loss so far: 2.729
(Epoch 29, iter 350/787) Average loss so far: 2.730
(Epoch 29, iter 400/787) Average loss so far: 2.714
(Epoch 29, iter 450/787) Average loss so far: 2.725
(Epoch 29, iter 500/787) Average loss so far: 2.711
(Epoch 29, iter 550/787) Average loss so far: 2.733
(Epoch 29, iter 600/787) Average loss so far: 2.737
(Epoch 29, iter 650/787) Average loss so far: 2.723
(Epoch 29, iter 700/787) Average loss so far: 2.724
(Epoch 29, iter 750/787) Average loss so far: 2.724
Average epoch loss: 2.725
This epoch took 7.99

7it [00:01,  5.51it/s]


validation loss: 3.1326404299054826


100%|██████████| 793/793 [00:01<00:00, 595.70it/s]

BLEU score: 0.055078340546709854, METEOR score: 0.20826842611097787


# Implementation of Extension 1 (GLoVE pretrained embeddings)

Please download [pretrained embeddings](https://huggingface.co/stanfordnlp/glove/resolve/main/glove.840B.300d.zip) and extract the contents to the project folder.

In [ ]:
def create_pretrained_embedding_dict(emb_filepath, **kwargs):
    ## build embedding dictionary
    pretrained_embedding_dict = dict()
    with open(emb_filepath, "r", **kwargs) as f:
        for line in f:
            wi, embi = line.rstrip().split(" ", 1)
            pretrained_embedding_dict[wi] = torch.from_numpy(np.fromstring(embi, sep=" "))
    return pretrained_embedding_dict

pretrained_embedding_dict = create_pretrained_embedding_dict("./glove.840B.300d.txt")

In [ ]:
reset_rng()
embedding_size=300
encoder_pretrained_embed = EncoderRNN(
    input_size=vocab.n_unique_words, embedding_size=embedding_size, hidden_size=HIDDEN_SIZE, 
    padding_value=vocab.word2index(PAD_WORD), pretrained_embedding_dict=pretrained_embedding_dict, 
    vocab=vocab).to(DEVICE)
# in the training script, decoder is always fed a non-end token and thus never needs to generate padding
# also it should never generate "<UNKNOWN>"
decoder_pretrained_embed = AttnDecoderRNN(
    embedding_size=embedding_size,hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-1,
    padding_val=vocab.word2index(PAD_WORD), dropout=DROPOUT, global_max_ing_len=MAX_INGR_LEN,
    pretrained_embedding_dict=pretrained_embedding_dict, vocab=vocab).to(DEVICE)

In [ ]:
initial_lr=1e-3
min_lr = 1e-5
n_epochs = 30
batch_size=128
encoder_pretrained_embed_optimizer = optim.Adam(encoder_pretrained_embed.parameters(), lr=initial_lr)
decoder_pretrained_embed_optimizer = optim.Adam(decoder_pretrained_embed.parameters(), lr=initial_lr)
# enc_scheduler = CosineAnnealingLR(encoder_pretrained_embed_optimizer, T_max=n_epochs, eta_min=min_lr)
# dec_scheduler = CosineAnnealingLR(decoder_pretrained_embed_optimizer, T_max=n_epochs, eta_min=min_lr)
enc_pretrained_embed_scheduler = CosineAnnealingLR(encoder_pretrained_embed_optimizer, T_max=n_epochs, eta_min=min_lr)
dec_pretrained_embed_scheduler = CosineAnnealingLR(decoder_pretrained_embed_optimizer, T_max=n_epochs, eta_min=min_lr)
identifier="pretrained_emb_attn_adam_without_intermediate_tags_wd0_lr1e-3"

pre_epoch_losses, pre_val_epoch_losses, pre_log = train(
    encoder_pretrained_embed, decoder_pretrained_embed, encoder_pretrained_embed_optimizer, decoder_pretrained_embed_optimizer, 
    train_ds, n_epochs=n_epochs, vocab=vocab, decoder_mode="attention", batch_size=batch_size, 
    enc_lr_scheduler=enc_pretrained_embed_scheduler, dec_lr_scheduler=dec_pretrained_embed_scheduler, 
    dev_ds_val_loss = dev_ds_val_loss, dev_ds_val_met=dev_ds_val_met, identifier=identifier,
    verbose_iter_interval=50)

save_log(identifier, pre_log, encoder_pretrained_embed_optimizer, decoder_pretrained_embed_optimizer, 
         enc_pretrained_embed_scheduler, dec_pretrained_embed_scheduler)

Starting epoch 1/30, enc lr scheduler: [0.001], dec lr scheduler: [0.001]
(Epoch 0, iter 50/787) Average loss so far: 6.968
(Epoch 0, iter 100/787) Average loss so far: 6.040
(Epoch 0, iter 150/787) Average loss so far: 5.995
(Epoch 0, iter 200/787) Average loss so far: 5.955
(Epoch 0, iter 250/787) Average loss so far: 5.910
(Epoch 0, iter 300/787) Average loss so far: 5.859
(Epoch 0, iter 350/787) Average loss so far: 5.839
(Epoch 0, iter 400/787) Average loss so far: 5.759
(Epoch 0, iter 450/787) Average loss so far: 5.627
(Epoch 0, iter 500/787) Average loss so far: 5.404
(Epoch 0, iter 550/787) Average loss so far: 5.209
(Epoch 0, iter 600/787) Average loss so far: 5.001
(Epoch 0, iter 650/787) Average loss so far: 4.851
(Epoch 0, iter 700/787) Average loss so far: 4.692
(Epoch 0, iter 750/787) Average loss so far: 4.582
Average epoch loss: 5.528
This epoch took 8.08877116839091 mins. Time remaining: 3.0 hrs 54.0 mins.


7it [00:01,  5.60it/s]


validation loss: 4.4685172353472025


100%|██████████| 793/793 [00:09<00:00, 87.13it/s] 


BLEU score: 0.0031624657072353694, METEOR score: 0.0796926100408187
Starting epoch 2/30, enc lr scheduler: [0.0009972883382072953], dec lr scheduler: [0.0009972883382072953]
(Epoch 1, iter 50/787) Average loss so far: 4.410
(Epoch 1, iter 100/787) Average loss so far: 4.295
(Epoch 1, iter 150/787) Average loss so far: 4.269
(Epoch 1, iter 200/787) Average loss so far: 4.187
(Epoch 1, iter 250/787) Average loss so far: 4.128
(Epoch 1, iter 300/787) Average loss so far: 4.080
(Epoch 1, iter 350/787) Average loss so far: 4.027
(Epoch 1, iter 400/787) Average loss so far: 3.997
(Epoch 1, iter 450/787) Average loss so far: 3.949
(Epoch 1, iter 500/787) Average loss so far: 3.910
(Epoch 1, iter 550/787) Average loss so far: 3.871
(Epoch 1, iter 600/787) Average loss so far: 3.844
(Epoch 1, iter 650/787) Average loss so far: 3.814
(Epoch 1, iter 700/787) Average loss so far: 3.778
(Epoch 1, iter 750/787) Average loss so far: 3.759
Average epoch loss: 4.009
This epoch took 8.075345408916473 mi

...

BLEU score: 0.054989968499987295, METEOR score: 0.21584912823420402
Starting epoch 30/30, enc lr scheduler: [1.2711661792704668e-05], dec lr scheduler: [1.2711661792704668e-05]
(Epoch 29, iter 50/787) Average loss so far: 2.667
(Epoch 29, iter 100/787) Average loss so far: 2.655
(Epoch 29, iter 150/787) Average loss so far: 2.648
(Epoch 29, iter 200/787) Average loss so far: 2.672
(Epoch 29, iter 250/787) Average loss so far: 2.658
(Epoch 29, iter 300/787) Average loss so far: 2.662
(Epoch 29, iter 350/787) Average loss so far: 2.664
(Epoch 29, iter 400/787) Average loss so far: 2.649
(Epoch 29, iter 450/787) Average loss so far: 2.657
(Epoch 29, iter 500/787) Average loss so far: 2.646
(Epoch 29, iter 550/787) Average loss so far: 2.667
(Epoch 29, iter 600/787) Average loss so far: 2.670
(Epoch 29, iter 650/787) Average loss so far: 2.655
(Epoch 29, iter 700/787) Average loss so far: 2.656
(Epoch 29, iter 750/787) Average loss so far: 2.661
Average epoch loss: 2.658
This epoch took 7.

7it [00:01,  6.06it/s]


validation loss: 3.0425445352281844


100%|██████████| 793/793 [00:01<00:00, 619.58it/s]


BLEU score: 0.057802547829115924, METEOR score: 0.21431412239195574


# Implementation of Extension 2 (Multi-Layer Encoder-Decoder + Neurologic Decoding)

### Defining Neurologic Decoding

In [ ]:
def repeat_for_k(x, k, dim=1):
    dims = [1]*x.ndim
    dims[dim] = k
    return x.repeat(dims)

def detect_negative_constraint(num_ks, curr_decoder_outs_shape, all_decoder_outs, neg_constraints):
    """_summary_

    Args:
        num_ks (int): number of hypothesis which have not ended
        curr_decoder_outs_shape: shape of current decoder output [k, |V|-1]
        all_decoder_outs (list): [k, |generation|]
        neg_constraints (List[List[int]]): list of negative constraints, nested list because constraints can be multi-word
    """

    # >1 if neg constraint satisfied, 0 otherwise
    # >1 makes sense if multiple negative constraints are satisfied (yielding larger penalty)
    neg_constraint_satisfied = torch.zeros(curr_decoder_outs_shape, device=DEVICE) # [k, |V|-1]

    for ki in range(num_ks):
        for neg_cons in neg_constraints:
            neg_constraint_exists = True
            neg_idx = neg_cons[-1]

            if (len(neg_cons) - 1) > len(all_decoder_outs[ki]):
                continue # i.e. neg_constraint_exists=False

            # to check if a negative constraint is satisfied (irreversible unsatisfaction) 
            # so need to check from back to front (only relevant for multi-word constraints)
            for word_idx, constraint_word in enumerate(neg_cons[:-1][::-1]):
                # if mismatch, then neg constraint is not satisfied
                if all_decoder_outs[ki][-(word_idx+1)] != constraint_word:
                    neg_constraint_exists = False
                    break

            neg_constraint_satisfied[ki][neg_idx] += neg_constraint_exists
    return neg_constraint_satisfied

def detect_low_likelihood(alpha, likelihood):
    """Detect likelihoods < top-alpha

    Args:
        alpha (_type_): _description_
        likelihood (_type_): [k, |V|-1]
    """
    # get the minimum value to be included within the top-alpha
    likelihood_penalty_thresh = likelihood.flatten().topk(alpha).values.min()

    return likelihood < likelihood_penalty_thresh

def update_irreversible_satisfaction(num_ks, k_irreversible_satisfaction, all_decoder_outs, pos_constraints,
                                   out_size):
    """_summary_

    Args:
        num_ks (int): number of hypothesis which have not ended
        beta (_type_): top-beta number of irreversible satisfactions to keep
        k_irreversible_satisfaction (_type_): number of irreversible satisfactions per hypothesis; [k]
        all_decoder_outs (_type_): list of previous generations for each hypothesis [k, |generation|]
        pos_constraints (List[List[List[int]]]): 3D list of shape [max_k, num positive constraints, length of positive constraint]
        out_size: dimension of generations |Vocab|-1
    """
    # [k, |V|-1]
    k_irreversible_satisfaction_now = k_irreversible_satisfaction[:, None].repeat(1, out_size)
    pos_constraints_satisfied = torch.full_like(k_irreversible_satisfaction_now, -1)

    for ki in range(num_ks):
        for pos_cons_idx, pos_cons in enumerate(pos_constraints[ki]):
            pos_constraint_exist = True
            
            ## similar to detecting irreversible unsatisfaction, 
            # we check from last (current) to first (previous generated text)
            pos_idx = pos_cons[-1]

            if (len(pos_cons) - 1) > len(all_decoder_outs[ki]):
                continue # i.e. pos_constraint_exist=False

            # checkds from 2nd last to first (only relevant for multi-word constraints)
            for word_idx, constraint_word in enumerate(pos_cons[:-1][::-1]): 
                if all_decoder_outs[ki][-(word_idx+1)] != constraint_word:
                    pos_constraint_exist = False
                    break

            if pos_constraint_exist:
                k_irreversible_satisfaction_now[ki][pos_idx] += 1
                pos_constraints_satisfied[ki][pos_idx] = pos_cons_idx
                
    return k_irreversible_satisfaction_now, pos_constraints_satisfied

def detect_low_irreversible_satisfactions(k_irreversible_satisfactions_now, # [k, |V|-1]
                                          beta):
    # get the minimum number of satisfied clauses to be included within the top-beta
    # need to use unique because many candidates can have the same number of satisfied clauses
    unique_num_irreversible_satisfactions = k_irreversible_satisfactions_now.flatten().unique()
    satisfaction_penalty_thresh = unique_num_irreversible_satisfactions[-min(beta, len(unique_num_irreversible_satisfactions))].item()
    return k_irreversible_satisfactions_now < satisfaction_penalty_thresh

def get_proportion_completion_reward(num_ks,
                                     scores, # [k, |V|-1]
                                     pos_constraints, # List[List[List[int]]]; [k, num constraints, len constraint]
                                     all_decoder_outs, # List [k, |generation|]
                                     lam=0.75
                                     ):
    reward = torch.zeros_like(scores)

    for ki in range(num_ks):
        for pos_cons in pos_constraints[ki]:
            ## just like in the paper, we also reward partial completion (reversible satisfaction)
            # to do this we need to do constraint prefix comparison for lengths i=0...|constraint|
            # because if a constraint is: [0, 1, 2, 3], a partial completion could be [0], [0,1], [0,1,2]
            # with full completion: [0,1,2,3]
            for word_idx, constraint_word in enumerate(pos_cons):
                if word_idx > len(all_decoder_outs[ki]):
                    break
                if word_idx == 0 or all_decoder_outs[ki][-word_idx:] == pos_cons[:word_idx]:
                    reward[ki][constraint_word] = max((word_idx+1) / len(pos_cons), reward[ki][constraint_word])
    
    return lam * reward

def neurologic_decoding(decoder, decoder_hidden, decoder_cell, encoder_houts,
                            ingredients, max_recipe_len,
                            pos_constraints, neg_constraints, k, alpha, beta,
                            neg_constraint_penalty, likelihood_penalty, low_irr_satisfaction_penalty, lam,
                            decoder_mode="attention"):
    """Neurological decoding for a particular sample in batch.

    Args:
        decoder (_type_): _description_
        decoder_hidden (_type_): [1, N=1, H]
        decoder_cell (_type_): [1, N=1, H]
        encoder_houts (_type_): [N=1, L_i, H]
        ingredients (_type_): [N=1, L_i]
        max_recipe_len (_type_): _description_
        pos_constraints (List[List[int]]): list of positive constraints, nested list because constraints can be multi-word 
                                           IMPORTANT: these are expected to be transformed to index using vocab
        neg_constraints (List[List[int]]): list of negative constraints, nested list because constraints can be multi-word
                                           IMPORTANT: these are expected to be transformed to index using vocab
        k (_type_): number of hypothesis per sample
        alpha (_type_): top-alpha likelihood which are not pruned
        beta (_type_): top-beta number of satisfied clauses which are not pruned
        neg_constraint_penalty (_type_): penalty for including negative constraint
        likelihood_penalty (_type_): penalty for not being in top-alpha likelihood
        low_irr_satisfaction_penalty (_type_): penalty for not being in top-beta no. of satisfied clauses
        lam (_type_): lambda to add constraint progress to score
        decoder_mode (str, optional): _description_. Defaults to "basic".
    """
    assert decoder_mode == "attention", "best model is attention, should be using attention!"

    K = torch.tensor([0]).to(DEVICE) # start with 1 hypothesis
    
    all_decoder_outs = [[SPECIAL_TAGS[REC_START]] for _ in range(k)] # stores decoder outputs for each hypothesis; [max_K]

    decoder_input = torch.full([1], SPECIAL_TAGS[REC_START], dtype=torch.long, device=DEVICE)

    # stores the running likelihoods for the k hypotheses
    k_likelihoods = torch.zeros([k], dtype=torch.float, device=DEVICE) # [max_K]

    # number of irreversible satisfactions so far for each hypothesis
    k_irreversible_satisfaction = torch.zeros_like(decoder_input) # [k]

    # lists *remaining* positive constraints for each hypotheses
    # once a positive constraint has been fully satisfied (irreversible satisfaction), it is removed
    # 3D list of shape [max_k, num positive constraints, length of positive constraint]
    # pos_constraints_i = [pos_constraints for _ in range(k)]
    pos_constraints_i = [pos_constraints]

    ## initialize inputs as the same for all ks because all of them have the same ingredients
    # encoder_houts_i = repeat_for_k(encoder_houts, k, dim=0) # [N=max_K, L_i, H]
    # decoder_hidden_i = repeat_for_k(decoder_hidden, k, dim=1) # [1, N=max_K, H]
    # decoder_cell_i = repeat_for_k(decoder_cell, k, dim=1) # [1, N=max_K, H]
    # ingredients_i = repeat_for_k(ingredients, k, dim=0) # [N=max_K, L_i]
    encoder_houts_i = encoder_houts
    decoder_hidden_i = decoder_hidden
    decoder_cell_i = decoder_cell
    ingredients_i = ingredients

    for recipe_i in range(max_recipe_len - 1): # generations are bounded by max length (-1 because of EOS)
        ## precondition: K is the list of hypotheses which have not ended

        num_ks = len(K) # some hypotheses can finish early so need to udpate this every iter
        valid_all_decoder_outs = [all_decoder_outs[i] for i in K]

        ## attention
        # decoder_out: log probs [k, |Vocab|-1]
        decoder_out, decoder_hidden_i, decoder_cell_i, attn_weights_i = decoder(
            decoder_input, decoder_hidden_i, decoder_cell_i, encoder_houts_i, ingredients_i
        )

        # sum all log probs with running log probs
        # [k, |V|-1] + [k, 1] = [k, |V|-1]
        likelihood_i = decoder_out + k_likelihoods[K].unsqueeze(-1)

        scores = decoder_out.clone() # used for selection (can no longer be interpreted as probabilities so we preserve likelihoods)

        ############# PRUNING #############

        # detect generations which will cause irreversible unsatisfaction; [k, |V|-1]
        neg_constraint_satisfied = detect_negative_constraint(
            num_ks, likelihood_i.shape, valid_all_decoder_outs, neg_constraints)
        
        # detect generations with low likelihood; [k, |V|-1]
        low_likelihoods = detect_low_likelihood(alpha, likelihood_i)

        # get potential total irreversible satisfaction (including already satisfied clauses) for each candidate
        # k_irreversible_satisfaction_now: [k, |V|-1]
        # pos_constraints_satisfied: [k, |V|-1]
        k_irreversible_satisfaction_now, pos_constraints_satisfied = update_irreversible_satisfaction(
            num_ks, k_irreversible_satisfaction, valid_all_decoder_outs, pos_constraints_i, 
            out_size=likelihood_i.size(-1))
        
        # detect generations with < top-beta number of irreversibly satisfied clauses; [k, |V|-1]
        low_irreversible_satisfaction = detect_low_irreversible_satisfactions(
            k_irreversible_satisfaction_now, beta)
        
        # perform soft pruning, i.e. penalizing instead of filtering out (see report)
        penalties = neg_constraint_satisfied * neg_constraint_penalty + \
                    low_likelihoods * likelihood_penalty + \
                    low_irreversible_satisfaction * low_irr_satisfaction_penalty
        
        scores -= penalties
        
        ############# SELECTION #############

        # get rewards for partial/full completion
        rewards = get_proportion_completion_reward(num_ks, scores, pos_constraints_i, valid_all_decoder_outs, lam=lam)

        scores += rewards # [k, |V|-1]

        # select top-k based on scores across all candidates
        topk_scores, topk_inds = scores.flatten().topk(num_ks if recipe_i > 0 else k)
        k_origin = torch.div(topk_inds, scores.size(-1), rounding_mode="floor")
        k_origin_global = K[k_origin] # get global k
        word_idx = topk_inds % scores.size(-1) # k top words

        prev_all_decoder_outs = deepcopy(all_decoder_outs)
        for wi, (ki, k_glob) in enumerate(zip((K if recipe_i > 0 else range(3)), k_origin_global)):
            all_decoder_outs[ki] = prev_all_decoder_outs[k_glob] + [word_idx[wi].item()]
            # all_decoder_outs[k_glob].append(word_idx[k_glob].item())

        k_likelihoods[K if recipe_i > 0 else range(k)] = likelihood_i[k_origin, word_idx] # [k]

        pos_constraints_chosen = [pos_constraints_i[ki] for ki in k_origin]

        for i, (ki, wi) in enumerate(zip(k_origin, word_idx)):
            # remove irreversibly satisfied constraint
            pos_constraints_chosen[i] = [c for cidx, c in enumerate(pos_constraints_chosen[i])
                                         if cidx != pos_constraints_satisfied[ki, wi]]
        pos_constraints_i = pos_constraints_chosen

        ############# PREPARE FOR NEXT ITERATION #############

        ## check if any of the hypotheses has ended (update K)
        not_eor = word_idx != SPECIAL_TAGS[REC_END]
        K = torch.arange(k, device=DEVICE)[K][not_eor] if recipe_i > 0 \
            else torch.arange(k, device=DEVICE)[not_eor]
        # K=K[not_eor] if recipe_i > 0 else torch.arange(k)[not_eor]
        # K=torch.arange(len(K) if recipe_i > 0 else k)[not_eor]

        if len(K) < 1:
            break

        ## postcondition: K is the list of hypotheses which have not ended

        ## determine inputs for next iter

        k_irreversible_satisfaction = k_irreversible_satisfaction_now[k_origin[not_eor], word_idx[not_eor]] # [k]
        decoder_input = word_idx[not_eor] # [k]

        ## all the same so no need to index using K
        if recipe_i == 0:
            encoder_houts_i = repeat_for_k(encoder_houts, len(K), dim=0)
            ingredients_i = repeat_for_k(ingredients, len(K), dim=0)
        else:
            encoder_houts_i = encoder_houts_i[:len(K)] # [N=k, L_i, H]
            ingredients_i = ingredients_i[:len(K)]

        ## different based on k
        decoder_hidden_i = decoder_hidden_i[:, k_origin[not_eor]]
        decoder_cell_i = decoder_cell_i[:, k_origin[not_eor]]
        pos_constraints_i = [pos_constraints_i[i] for i in not_eor.nonzero().flatten()]
    else:
        for ki in K.tolist():
            all_decoder_outs[ki].append(SPECIAL_TAGS[REC_END])

    k_likelihoods_normalized = k_likelihoods / torch.tensor([len(o) for o in all_decoder_outs],
                                                            device=k_likelihoods.device)

    return all_decoder_outs, k_likelihoods_normalized

def eval_neuro_decoding_iter(ingredients, ing_lens, encoder, decoder, vocab, pos_constraints, neg_constraints,
                              max_recipe_len=MAX_RECIPE_LEN,
                              **kwargs):
    assert encoder.training is False and decoder.training is False

    enc_out, enc_out_lens, enc_h_final, enc_c_final = encoder(ingredients, ing_lens)
    
    # initialize decoder hidden state as final encoder hidden state
    decoder_hidden = enc_h_final
    decoder_cell = enc_c_final

    # all_decoder_outs (List[List[int]]): list of k hypotheses
    # k_likelihoods_normalized (tensor): has overall likelihood of each hypotheses shape [k]
    all_decoder_outs, k_likelihoods_normalized = neurologic_decoding(decoder, decoder_hidden, decoder_cell, enc_out,
                                                                     ingredients, max_recipe_len, pos_constraints,
                                                                     neg_constraints, **kwargs)
    
    max_i = k_likelihoods_normalized.argmax()
    final_decoder_out = all_decoder_outs[max_i]

    final_decoder_out_txt = [vocab.index2word[w] for w in final_decoder_out]

    return final_decoder_out_txt

def build_constraints_dict(all_ingredients_list, vocab):
    constraints_dict = {}
    valid_ingredients_list = deepcopy(all_ingredients_list)
    invalid_ingredients = []
    for ingredient in valid_ingredients_list:
        invalid_ingredient=False
        ingredient_idx_form = []

        for word in ingredient.split():
            if not vocab.word_exist_in_vocab(word):
                invalid_ingredient = True
                break
            ingredient_idx_form.append(vocab.word2index(word))
        if not invalid_ingredient:
            constraints_dict[ingredient] = ingredient_idx_form
        else:
            # valid_ingredients_list.remove(ingredient)
            invalid_ingredients.append(ingredient)
    return constraints_dict, invalid_ingredients

def eval_neuro_decoding(encoder, decoder, dataset, vocab, all_ingredients_list, 
                        max_recipe_len=MAX_RECIPE_LEN, **kwargs):
    
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False, 
                                              collate_fn=pad_collate(vocab, train=False))
    
    all_decoder_outs = [] # (List[List[str]]): List of len `N`, each element is the generated sequence for that sample
    all_gt_recipes = [] # (List[List[str]])
    all_ingredients = []

    constraints_dict, invalid_ingredients = build_constraints_dict(all_ingredients_list, vocab)

    valid_ingredients_list = [i for i in all_ingredients_list if i not in invalid_ingredients]

    all_ingredients_regex = get_ingredients_regex(valid_ingredients_list)

    encoder.eval()
    decoder.eval()

    with torch.no_grad():
        for ingredients, recipes, ing_lens, _ in tqdm(dataloader):
            pos_constraints, neg_constraints, ingredients_text = create_pos_neg_constraints(
                ingredients, vocab, all_ingredients_regex, constraints_dict)
            final_decoder_out_txt = eval_neuro_decoding_iter(
                ingredients, ing_lens, encoder, decoder, vocab, pos_constraints, neg_constraints,
                max_recipe_len, **kwargs
            )
            all_decoder_outs += [final_decoder_out_txt]
            all_gt_recipes += recipes
            all_ingredients += [ingredients_text]

    return all_decoder_outs, all_gt_recipes, all_ingredients

def create_pos_neg_constraints(ingredients_idxs, # expecting batch size 1
                               vocab,
                               all_ingredients_regex,
                               constraints_dict):
    
    ingredients_text = " ".join([vocab.index2word[i] for i in ingredients_idxs[0].tolist()])
    input_ingredients = find_ingredients_in_text(ingredients_text, all_ingredients_regex)
    input_ingrs_partial = ' '.join(input_ingredients).split()
    pos_constraints = [constraints_dict[ing] for ing in sorted(list(input_ingredients), key=len)]
    neg_constraints = []
    for constraint in constraints_dict.keys():
        if constraint in pos_constraints:
            continue
        valid_negative_constraint=True
        for word in constraint.split():
            if word in input_ingrs_partial:
                valid_negative_constraint = False
                break
        if valid_negative_constraint:
            neg_constraints.append(constraints_dict[constraint])
    return pos_constraints, neg_constraints, ingredients_text

In [ ]:
reset_rng()
embedding_size=300
num_layers=3
encoder_multilayer_attn = EncoderRNN(vocab.n_unique_words, embedding_size=embedding_size, hidden_size=HIDDEN_SIZE, 
                          padding_value=vocab.word2index(PAD_WORD), num_lstm_layers=num_layers).to(DEVICE)
# in the training script, decoder is always fed a non-end token and thus never needs to generate padding
# also it should never generate "<UNKNOWN>"
# decoder = DecoderRNN(embedding_size=embedding_size,hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-2).to(DEVICE)
decoder_multilayer_attn = AttnDecoderRNN(embedding_size, hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-1, 
                              padding_val=vocab.word2index(PAD_WORD), dropout=DROPOUT, num_lstm_layers=num_layers).to(DEVICE)

In [ ]:
initial_lr=1e-3
min_lr = 1e-5
n_epochs = 30
batch_size=128
encoder_multilayer_attn_optimizer = optim.Adam(encoder_multilayer_attn.parameters(), lr=initial_lr)
decoder_multilayer_attn_optimizer = optim.Adam(decoder_multilayer_attn.parameters(), lr=initial_lr)
enc_multilayer_attn_scheduler = CosineAnnealingLR(encoder_multilayer_attn_optimizer, T_max=n_epochs, eta_min=min_lr)
dec_multilayer_attn_scheduler = CosineAnnealingLR(decoder_multilayer_attn_optimizer, T_max=n_epochs, eta_min=min_lr)
identifier="multilayer_attn_adam_without_intermediate_tags_wd0_lr1e-3"

multilayer_attn_epoch_losses, multilayer_attn_val_epoch_losses, multilayer_attn_log = train(
    encoder_multilayer_attn, decoder_multilayer_attn, encoder_multilayer_attn_optimizer, decoder_multilayer_attn_optimizer, train_ds, 
    n_epochs=n_epochs, vocab=vocab, decoder_mode="attention", batch_size=batch_size, 
    enc_lr_scheduler=enc_multilayer_attn_scheduler, dec_lr_scheduler=dec_multilayer_attn_scheduler, 
    dev_ds_val_loss = dev_ds_val_loss, dev_ds_val_met=dev_ds_val_met, identifier=identifier,
    verbose_iter_interval=50)

save_log(identifier, multilayer_attn_log, encoder_multilayer_attn_optimizer, decoder_multilayer_attn_optimizer, 
         enc_multilayer_attn_scheduler, dec_multilayer_attn_scheduler)

Starting epoch 1/30, enc lr scheduler: [0.001], dec lr scheduler: [0.001]
(Epoch 0, iter 50/787) Average loss so far: 6.988
(Epoch 0, iter 100/787) Average loss so far: 6.058
(Epoch 0, iter 150/787) Average loss so far: 6.035
(Epoch 0, iter 200/787) Average loss so far: 6.032
(Epoch 0, iter 250/787) Average loss so far: 6.019
(Epoch 0, iter 300/787) Average loss so far: 6.011
(Epoch 0, iter 350/787) Average loss so far: 6.015
(Epoch 0, iter 400/787) Average loss so far: 6.006
(Epoch 0, iter 450/787) Average loss so far: 6.002
(Epoch 0, iter 500/787) Average loss so far: 5.990
(Epoch 0, iter 550/787) Average loss so far: 5.972
(Epoch 0, iter 600/787) Average loss so far: 5.961
(Epoch 0, iter 650/787) Average loss so far: 5.934
(Epoch 0, iter 700/787) Average loss so far: 5.930
(Epoch 0, iter 750/787) Average loss so far: 5.931
Average epoch loss: 6.053
This epoch took 8.784394307931263 mins. Time remaining: 4.0 hrs 14.0 mins.


7it [00:01,  5.15it/s]


validation loss: 5.933405944279262


100%|██████████| 7/7 [00:08<00:00,  1.15s/it]
/home/junn/miniconda3/envs/nlp/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
100%|██████████| 793/793 [00:06<00:00, 114.02it/s]


BLEU score: 5.8211892454135815e-80, METEOR score: 0.034986121334209475
Starting epoch 2/30, enc lr scheduler: [0.0009972883382072953], dec lr scheduler: [0.0009972883382072953]
(Epoch 1, iter 50/787) Average loss so far: 5.916
(Epoch 1, iter 100/787) Average loss so far: 5.902
(Epoch 1, iter 150/787) Average loss so far: 5.896
(Epoch 1, iter 200/787) Average loss so far: 5.880
(Epoch 1, iter 250/787) Average loss so far: 5.882
(Epoch 1, iter 300/787) Average loss so far: 5.891
(Epoch 1, iter 350/787) Average loss so far: 5.881
(Epoch 1, iter 400/787) Average loss so far: 5.874
(Epoch 1, iter 450/787) Average loss so far: 5.868
(Epoch 1, iter 500/787) Average loss so far: 5.830
(Epoch 1, iter 550/787) Average loss so far: 5.730
(Epoch 1, iter 600/787) Average loss so far: 5.635
(Epoch 1, iter 650/787) Average loss so far: 5.547
(Epoch 1, iter 700/787) Average loss so far: 5.400
(Epoch 1, iter 750/787) Average loss so far: 5.205
Average epoch loss: 5.724
This epoch took 8.680376549561819

...

BLEU score: 0.053924556203878635, METEOR score: 0.2136804902885653
Starting epoch 30/30, enc lr scheduler: [1.2711661792704668e-05], dec lr scheduler: [1.2711661792704668e-05]
(Epoch 29, iter 50/787) Average loss so far: 2.496
(Epoch 29, iter 100/787) Average loss so far: 2.494
(Epoch 29, iter 150/787) Average loss so far: 2.521
(Epoch 29, iter 200/787) Average loss so far: 2.511
(Epoch 29, iter 250/787) Average loss so far: 2.510
(Epoch 29, iter 300/787) Average loss so far: 2.511
(Epoch 29, iter 350/787) Average loss so far: 2.505
(Epoch 29, iter 400/787) Average loss so far: 2.493
(Epoch 29, iter 450/787) Average loss so far: 2.497
(Epoch 29, iter 500/787) Average loss so far: 2.521
(Epoch 29, iter 550/787) Average loss so far: 2.491
(Epoch 29, iter 600/787) Average loss so far: 2.503
(Epoch 29, iter 650/787) Average loss so far: 2.506
(Epoch 29, iter 700/787) Average loss so far: 2.509
(Epoch 29, iter 750/787) Average loss so far: 2.515
Average epoch loss: 2.506
This epoch took 8.6

7it [00:01,  5.22it/s]


validation loss: 3.0682195595332553


100%|██████████| 793/793 [00:01<00:00, 777.01it/s]

BLEU score: 0.05360497700665978, METEOR score: 0.21295206921320584


# Quantitative Evaluation

In [ ]:
def get_prop_input_num_extra_ingredients_m(ingr_txt, recipe_txt, all_ings_regex, verbose=False):
    """get proportion of input ingredients and number of extra ingredients in recipe (`txt`).

    Args:
        ingr_txt (str): ingredients text
        recipe_txt (str): recipe text
        all_ings_regex (str): regex to match all valid ingredients
        input_ingredients_iter (List or Set): iterable of unique ingredients list
    """
    # get set of input ingredients
    input_ingredients = find_ingredients_in_text(ingr_txt, all_ings_regex)
    # # get regex to match only input ingredients
    # input_ingredients_regex = get_ingredients_regex(input_ingredients)

    all_ings_in_text = [i.strip('<>') for i in set(re.findall("<[A-Za-z ]+>", recipe_txt))]
    input_ings_in_text = []
    extra_ings_in_text = []
    for ing in all_ings_in_text:
        if ing in input_ingredients:
            input_ings_in_text.append(ing)
        else:
            extra_ings_in_text.append(ing)
    if verbose:
        print(f"=====Input ingredients in text=====\n{input_ings_in_text}")
        print(f"\n=====All ingredients in text===== \n{all_ings_in_text}")

    prop_input_ings = len(input_ings_in_text) / len(input_ingredients)
    num_extra_ings = len(extra_ings_in_text)
    return prop_input_ings, num_extra_ings

def get_included_extra_ingrs_single(ingredients_i: str, recipe_i: str, 
                                    all_ingredients_regex:str, invalid_ingredients_regex:str,
                                     partial_match=False):
    ## get all ingredients in input ingredients
    input_ingrs = find_ingredients_in_text(ingredients_i, all_ingredients_regex, enforce_unique=True)

    # some data samples have ingredients which are not recognized by the regex
    if len(input_ingrs) < 1:
        return None, None
    
    ## get all ingredients in recipe
    all_recipe_ingrs = find_ingredients_in_text(recipe_i, all_ingredients_regex, enforce_unique=False)
    # get all invalid ingredients in recipe (start of sentence, etc.)
    invalid_recipe_ingrs = find_ingredients_in_text(recipe_i, invalid_ingredients_regex, enforce_unique=False)

    # remove a single instance of the invalid ingredient
    for i in range(len(invalid_recipe_ingrs)):
        if type(invalid_recipe_ingrs[i]) == tuple:
            invalid_w = invalid_recipe_ingrs[i][-1]
        else:
            invalid_w = invalid_recipe_ingrs[i]
        all_recipe_ingrs.remove(invalid_w)

    valid_recipe_ingrs = set(all_recipe_ingrs) # remove duplicates

    included_ingrs = 0
    for recipe_ingr in valid_recipe_ingrs:
        ## whole match
        if recipe_ingr in input_ingrs:
            included_ingrs += 1
            continue
        ## partial match
        if partial_match:
            indiv_ingr_parts = " ".join(input_ingrs).split()
            if recipe_ingr in indiv_ingr_parts:
                included_ingrs += 1
                continue
    num_extra_ingrs = len(valid_recipe_ingrs) - included_ingrs

    prop_included_ingrs = min(1, included_ingrs/len(input_ingrs))

    return prop_included_ingrs, num_extra_ingrs

def get_prop_input_num_extra_ingredients(all_ingredients, all_recipes, 
                                         all_ingredients_regex, invalid_ingredients_regex, partial_match=True):
    prop_included_ingrs_all = []
    num_extra_ingrs_all = []
    for ingredients_i, recipe_i in tqdm(zip(all_ingredients, all_recipes)):
        prop_included_ingrs, num_extra_ingrs = get_included_extra_ingrs_single(
            ingredients_i, recipe_i, all_ingredients_regex, invalid_ingredients_regex, partial_match)
        if prop_included_ingrs is not None:
            prop_included_ingrs_all.append(prop_included_ingrs)
            num_extra_ingrs_all.append(num_extra_ingrs)
    avg_prop_included_ingrs = sum(prop_included_ingrs_all) / len(prop_included_ingrs_all)
    avg_num_extra_ingrs = sum(num_extra_ingrs_all) / len(num_extra_ingrs_all)
    return avg_prop_included_ingrs, avg_num_extra_ingrs

def convert_eval_out_to_get_ingredient_metrics(all_generated_recipes, all_gt_ingredients, vocab,
                                               all_ingredients_lst, skip_ing_processing=False):
    if skip_ing_processing:
        ingredient_txts = all_gt_ingredients
    else:
        ingredient_txts = []
        for ingredients in all_gt_ingredients:
            ing_text = " ".join([vocab.index2word[i] for i in ingredients
                            if i != vocab.word2index(PAD_WORD)])
            ingredient_txts += [ing_text]
    generated_recipes_concat = [" ".join(l) for l in all_generated_recipes]
    all_ingredient_regex = get_ingredients_regex(all_ingredients_lst)
    invalid_ingredient_regex = get_invalid_ingredients_regex(all_ingredients_lst)
    avg_prop_included_ingrs, avg_num_extra_ingrs = get_prop_input_num_extra_ingredients(
        ingredient_txts, generated_recipes_concat, all_ingredient_regex, invalid_ingredient_regex)
    print(f"Avg. % given ingredients: {avg_prop_included_ingrs*100:.3f}%\n"
          f"Avg. number of extra ingredients: {avg_num_extra_ingrs:.3f}")

In [ ]:
all_ingredients_lst = get_all_ingredients("./ingredient_set.json")

### Baseline 1

In [ ]:
load_model(encoder, decoder, "adam_without_intermediate_tags_with_val_wd0_lr1e-3_ep_24")
all_decoder_outs, all_gt_recipes, all_gt_ingredients = eval(
    encoder, decoder, test_ds, vocab, batch_size=128, decoder_mode="basic",
    max_recipe_len=MAX_RECIPE_LEN)

In [ ]:
calc_bleu(all_gt_recipes, all_decoder_outs)

In [ ]:
calc_meteor(all_gt_recipes, all_decoder_outs, split_gt=False)

In [ ]:
convert_eval_out_to_get_ingredient_metrics(all_decoder_outs, all_gt_ingredients,
                                           vocab, all_ingredients_lst)

### Baseline 2

In [ ]:
load_model(encoder_attn, decoder_attn, "attn_adam_without_intermediate_tags_wd0_lr1e-3_ep_26")
all_decoder_outs, all_gt_recipes, all_gt_ingredients = eval(
    encoder_attn, decoder_attn, test_ds, vocab, batch_size=128, decoder_mode="attention",
    max_recipe_len=MAX_RECIPE_LEN)

In [ ]:
calc_bleu(all_gt_recipes, all_decoder_outs)

0.04949963728618708

In [ ]:
calc_meteor(all_gt_recipes, all_decoder_outs, split_gt=False)

100%|██████████| 774/774 [00:01<00:00, 629.82it/s]


0.20855536635420194

In [ ]:
convert_eval_out_to_get_ingredient_metrics(all_decoder_outs, all_gt_ingredients,
                                           vocab, all_ingredients_lst)

774it [00:00, 1011.72it/s]

Avg. % given ingredients: 33.453%
Avg. number of extra ingredients: 2.295


### Extension 1

In [ ]:
load_model(encoder_pretrained_embed, decoder_pretrained_embed, 
           "pretrained_emb_attn_adam_without_intermediate_tags_wd0_lr1e-3_ep_29")
all_decoder_outs, all_gt_recipes, all_gt_ingredients = eval(
    encoder_pretrained_embed, decoder_pretrained_embed, test_ds, vocab, batch_size=128, 
    decoder_mode="attention", max_recipe_len=MAX_RECIPE_LEN)

In [ ]:
calc_bleu(all_gt_recipes, all_decoder_outs)

0.05519142942964554

In [ ]:
calc_meteor(all_gt_recipes, all_decoder_outs, split_gt=False)

100%|██████████| 774/774 [00:01<00:00, 576.27it/s]


0.2100281888663272

In [ ]:
convert_eval_out_to_get_ingredient_metrics(all_decoder_outs, all_gt_ingredients,
                                           vocab, all_ingredients_lst)

774it [00:00, 1022.21it/s]

Avg. % given ingredients: 31.074%
Avg. number of extra ingredients: 2.104


### Extension 2

#### Without Neurologic Decoding

In [ ]:
load_model(encoder_multilayer_attn, decoder_multilayer_attn, "multilayer_attn_adam_without_intermediate_tags_wd0_lr1e-3_ep_14")
all_decoder_outs_multi, all_gt_recipes_multi, all_gt_ingredients_multi = eval(
    encoder_multilayer_attn, decoder_multilayer_attn, test_ds, vocab, batch_size=128, decoder_mode="attention",
    max_recipe_len=MAX_RECIPE_LEN)

In [ ]:
calc_bleu(all_decoder_outs_multi, all_gt_recipes_multi)

0.053654397203939004

In [ ]:
calc_meteor(all_gt_recipes_multi, all_decoder_outs_multi, split_gt=False)

100%|██████████| 774/774 [00:01<00:00, 709.21it/s]


0.21415571426435778

In [ ]:
convert_eval_out_to_get_ingredient_metrics(all_decoder_outs_multi, all_gt_ingredients_multi,
                                           vocab, all_ingredients_lst)

774it [00:00, 1157.73it/s]

Avg. % given ingredients: 32.267%
Avg. number of extra ingredients: 2.223


#### With Neurologic Decoding

! WARNING: this code takes around 20-30 mins to run

In [ ]:
k=3
alpha=50 # likelihood
beta = 2 # num constraints satisfied
neg_constraint_penalty, likelihood_penalty, low_irr_satisfaction_penalty = 10, 0.5, 0.5
lam = 5.0

all_decoder_outs, all_gt_recipes, all_gt_ings = eval_neuro_decoding(
    encoder_multilayer_attn, decoder_multilayer_attn, test_ds, vocab, all_ingredients_lst, k=k, alpha=alpha, beta=beta, 
    neg_constraint_penalty=neg_constraint_penalty, likelihood_penalty=likelihood_penalty, 
    low_irr_satisfaction_penalty=low_irr_satisfaction_penalty, lam=lam)


100%|██████████| 774/774 [16:36<00:00,  1.29s/it]


In [ ]:
calc_bleu(all_gt_recipes, all_decoder_outs)

0.05810241560787664

In [ ]:
calc_meteor(all_gt_recipes, all_decoder_outs, split_gt=False)

100%|██████████| 774/774 [00:01<00:00, 712.33it/s]


0.23339527798724602

In [ ]:
convert_eval_out_to_get_ingredient_metrics(all_decoder_outs, all_gt_ings,
                                           vocab, all_ingredients_lst, skip_ing_processing=True)

774it [00:00, 1072.67it/s]

Avg. % given ingredients: 79.462%
Avg. number of extra ingredients: 0.274


# Metric Sample

In [ ]:
all_ings = get_all_ingredients("./ingredient_set.json")
all_ings_regex = get_ingredients_regex(all_ings)
metric_sample_ings, metric_sample_gold_recipe, metric_sample_generated_recipe = \
    load_metric_sample("./metric_sample.txt")

In [ ]:
prop_inp_ings, n_extra_ings = get_prop_input_num_extra_ingredients_m(
    metric_sample_ings, metric_sample_generated_recipe, all_ings_regex, verbose=True)
print(f"\nproportion of input ingredients: {prop_inp_ings}\nnumber of extra ingredients: {n_extra_ings}")

=====Input ingredients in text=====
['lemon juice', 'orange juice', 'water', 'sugar', 'strawberries']

=====All ingredients in text===== 
['lemon juice', 'orange juice', 'water', 'cantaloupe', 'vanilla ice cream', 'sugar', 'strawberries']

proportion of input ingredients: 1.0
number of extra ingredients: 2


In [ ]:
bleu_score = calc_bleu([metric_sample_gold_recipe], [metric_sample_generated_recipe], split_gt=True, split_gen=True)
meteor_score = calc_meteor([metric_sample_gold_recipe], [metric_sample_generated_recipe], split_gt=True, split_gen=True)
print(f"BLEU score: {bleu_score}, METEOR score: {meteor_score}")

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 458.90it/s]

BLEU score: 0.2757364975156813, METEOR score: 0.5479209577754892


# Qualitative Evaluation

In [ ]:
ingredients_sample_orig = "2 c sugar, 1/4 c lemon juice, 1 c water, 1/3 c orange juice, 8 c strawberries"

perform relevant preprocessing steps (note: these operations are taken from `preprocess_data` and is the same preprocessing done on the dataset)

In [ ]:
ingredients_sample = re.sub("([^0-9a-zA-Z.'\"/ ])", r" \1 ", ingredients_sample_orig)
ingredients_sample = '<INGREDIENT_START> ' + ingredients_sample + ' <INGREDIENT_END>'
ingredients_sample = re.sub('[ ]{2,}', " ", ingredients_sample)

In [ ]:
ingredients_sample

'<INGREDIENT_START> 2 c sugar , 1/4 c lemon juice , 1 c water , 1/3 c orange juice , 8 c strawberries <INGREDIENT_END>'

In [ ]:
ingredients_sample_idxs = torch.tensor([vocab.word2index(w) for w in ingredients_sample.split(" ")],
                                       dtype=torch.long, device=DEVICE)
# convert to batch form with batch size 1 as model is expecting batched input
ingredients_sample_idxs = ingredients_sample_idxs[None] 

In [ ]:
ingredients_sample_idxs

tensor([[   0,   22,   12,   16,   78,   32,   12,  641,  900,   78,   17,   12,
          311,   78,  235,   12,  791,  900,   78,  385,   12, 1218,    1]],
       device='cuda:0')

In [ ]:
ingr_lens = torch.tensor([len(x) for x in ingredients_sample_idxs], dtype=torch.long, device=DEVICE)

In [ ]:
ingr_lens

tensor([23], device='cuda:0')

In [ ]:
qual_res_basic = get_predictions_iter(ingredients_sample_idxs, ingr_lens, encoder, decoder,
                                        vocab)

In [ ]:
' '.join(qual_res_basic[0])

'<RECIPE_START> combine sugar , cornstarch and water in a saucepan bring to a boil , stirring constantly boil for 1 minute remove from heat and stir in lemon juice and lemon juice pour into a large bowl and stir in the orange juice and lemon juice pour into a large bowl and stir in the orange juice and lemon juice pour into a large bowl and chill <RECIPE_END>'

In [ ]:
qual_res_attn = get_predictions_iter(ingredients_sample_idxs, ingr_lens, encoder_attn, decoder_attn,
                                    vocab, decoder_mode="attention")

In [ ]:
' '.join(qual_res_attn[0])

'<RECIPE_START> combine sugar , water , lemon juice and salt in a saucepan bring to a boil , stirring constantly boil for 5 minutes , stirring constantly remove from heat and stir in lemon juice and pour into a sterilized jars seal and store in refrigerator <RECIPE_END>'

In [ ]:
qual_res_attn_pre_embed = get_predictions_iter(
    ingredients_sample_idxs, ingr_lens, encoder_pretrained_embed, decoder_pretrained_embed,
    vocab, decoder_mode="attention")

In [ ]:
' '.join(qual_res_attn_pre_embed[0])

'<RECIPE_START> combine sugar , water , and lemon juice in a saucepan bring to a boil over medium heat , stirring constantly , until sugar dissolves remove from heat and cool to room temperature add vanilla and mix well chill <RECIPE_END>'

In [ ]:
constraints_dict, invalid_ingredients = build_constraints_dict(all_ingredients_lst, vocab)
valid_ingredients_list = [i for i in all_ingredients_lst if i not in invalid_ingredients]
valid_ingredients_regex = get_ingredients_regex(valid_ingredients_list)

In [ ]:
pos_constraints, neg_constraints, ingredients_text = create_pos_neg_constraints(
    ingredients_sample_idxs, vocab, valid_ingredients_regex, constraints_dict
)

In [ ]:
pos_constraints

[[16], [311], [641, 900], [1218], [791, 900]]

In [ ]:
all_constraints_text = []
for l in pos_constraints:
    constraint_text = []
    for idx in l:
        constraint_text.append(vocab.index2word[idx])
    all_constraints_text.append(' '.join(constraint_text))

In [ ]:
all_constraints_text

['sugar', 'water', 'lemon juice', 'strawberries', 'orange juice']

In [ ]:
ingredients_text

'<INGREDIENT_START> 2 c sugar , 1/4 c lemon juice , 1 c water , 1/3 c orange juice , 8 c strawberries <INGREDIENT_END>'

In [ ]:
k=3
alpha=50 # likelihood
beta = 2 # num constraints satisfied
neg_constraint_penalty, likelihood_penalty, low_irr_satisfaction_penalty = 10, 0.5, 0.5
lam = 5.0

In [ ]:
final_decoder_out_txt = eval_neuro_decoding_iter(ingredients_sample_idxs, ingr_lens, encoder_multilayer_attn, decoder_multilayer_attn,
                         vocab, pos_constraints, neg_constraints, max_recipe_len=600, k=k, alpha=alpha, beta=beta,
                         neg_constraint_penalty=neg_constraint_penalty, likelihood_penalty=likelihood_penalty, 
                         low_irr_satisfaction_penalty=low_irr_satisfaction_penalty, lam=lam)

In [ ]:
' '.join(final_decoder_out_txt)

'<RECIPE_START> combine sugar and water in a saucepan and bring to a boil add orange juice , lemon juice , and sugar bring to a boil and boil for 5 minutes remove from heat and let cool slightly add strawberries and sugar and stir until dissolved pour into glasses , cover and chill <RECIPE_END>'